The goal of this Notebook will be evaluate the performance of each ANN1, ANN2, and each version of ANN3 and ANN4

In [ ]:
#First we will run the functions workbook 
%run Functions.ipynb

First, load in the experimental data, the parameters determined from the traditional least-square regression process on the completeEASE software, and the optical properties of the substrates and void.

In [ ]:
# Define path where optical properties are
os.chdir(r"XXXX") ########## PLEASE PUT THE DIRECTORY TO THE FOLDER LABELED "Optical Properties" in the XXXX space ####################
# File name for soda lime glass substrate
file =  "Junda_SLG.csv"
# Read in SLG optical properties
SLG = pd.read_csv(file)
SLG.name = 'SLG'

SLG['e'] = SLG['e1'].to_numpy() +  1j * SLG['e2'].to_numpy()


# File name for void
file =  "Void.csv"
# Read in void optical properties
Void = pd.read_csv(file)
Void.name = "Void"

# Define the photon energy range as variable E for future use. 
E = SLG['Energy (eV)']
# Define the same range in wavelength for future use
wv = SLG['Wavelength (nm)']

# File name for Si wafer from CompleteEASE database 
file =  "Si_JAW.csv"
# load in Si wafer
Si_JAW = pd.read_csv(file)
Si_JAW.name = "Si_JAW"

# File name for native oxide from CompleteEASE database 
file =  "NTVE_JAW.csv"
NTVE_JAW = pd.read_csv(file)
NTVE_JAW.name = "NTVE_JAW"


In [ ]:
# Now load in the experimental data and the associated LSR parameters for evaluation and comparisons

# Location of files
os.chdir(r"XXXX") ########## PLEASE PUT THE DIRECTORY TO THE FOLDER LABELED "Traditional_LSR" in the XXXX space ####################

############ MV1519 Sample Load in ############################

file = "MV1519-Data.csv" # File name for experimental data for sample MV1519
MV1519 = pd.read_csv(file) # read file as pandas dataframe
MV1519.name = 'MV1519' # name dataframe  
MV1519 =  MV1519.drop(MV1519.columns[0], axis=1) # Reformat data by removing the first column


SE_data = [] # list that will eventually contain the experimental data. Each entry in the list will be a pandas dataframe with SE data in  (E,N,C,S)

for i in range(81): # iterate for the 81 point map.
    
    temp = MV1519.iloc[:, (i*3):(3 *(i+1))] # read a single set of (N,C,S)
    temp.insert(0, 'Energy (eV)', E) # add Photon Enery E in eV
    SE_data.append(temp) #add data to list
    
MV1519 = np.array(SE_data) # reformat data to an np array.


# Load in the parameters from the LSR analysis done on CompleteEASE.
file = "MV1519-CL-Params.csv" # file with parameters from LSR
MV1519_params = pd.read_csv(file) # read in files and pandas dataframe

############ MV1523 Sample Load in ############################

file = "MV1523-Data.csv" # File name for experimental data for sample MV1523
MV1523 = pd.read_csv(file) # read file as pandas dataframe
MV1523.name = 'MV1523' # name file 
MV1523 =  MV1523.drop(MV1523.columns[0], axis=1) # Reformat data by removing the first column


SE_data = [] # list that will eventually contain the experimental data. Each entry in the list will be a pandas dataframe with SE data in  (E,N,C,S)

for i in range(81): # iterate for the 81 point map.
    
    temp = MV1523.iloc[:, (i*3):(3 *(i+1))] # read a single set of (N,C,S)
    temp.insert(0, 'Energy (eV)', E) # add Photon Enery E in eV
    SE_data.append(temp) #add data to list
    
MV1523 = np.array(SE_data) # reformat data to an np array.

# Load in the parameters from the LSR analysis done on CompleteEASE.
file = "MV1523-CL-Params.csv" # file with parameters from least-square analysis
MV1523_params = pd.read_csv(file) # read in files and pandas dataframe


############ MV1530 Sample Load in ############################

file = "MV1530-Data.csv" # File name for experimental data for sample MV1530
MV1530 = pd.read_csv(file) # read file as pandas dataframe
MV1530.name = 'MV1530' # name file 
MV1530 =  MV1530.drop(MV1530.columns[0], axis=1) # Reformat data by removing the first column


SE_data = [] # list that will eventually contain the experimental data. Each entry in the list will be a pandas dataframe with SE data in  (E,N,C,S)

for i in range(81): # iterate for the 81 point map.
    
    temp = MV1530.iloc[:, (i*3):(3 *(i+1))] # read a single set of (N,C,S)
    temp.insert(0, 'Energy (eV)', E)  # add Photon Enery E in eV
    SE_data.append(temp) #add data to list

    
MV1530 = np.array(SE_data) # reformat data to an np array.


# Now we want to load in the parameters from the least-square regression analysis done on CompleteEASE.
file = "MV1530-SLG_Interface-Params.csv" # file with parameters from least-square analysis
MV1530_params = pd.read_csv(file) # read in files and pandas dataframe


Now load in each ANN for evaluation. Starting with ANN1 and ANN2.

In [ ]:
#Load In Model

# move to location of the pre-trained models
os.chdir(r"XXXX") ########## PLEASE PUT THE DIRECTORY TO THE FOLDER LABELED "Models" in the XXXX space ####################

# Load in models that make boolean predictions on structure
ANN1 = load_model('ANN1.keras')
ANN2 = load_model('ANN2.keras')


ANN1 and ANN2 are trained with simulated spectroscopic ellipsometry data from Data_Set_3. The associated test set will be loaded in to evaluate the performance of ANN1 and ANN2. The test set contains simulated data generated in the same way, but this data is not used during taining.



In [ ]:
# Load in test data from Data_Set_3.

# Identify the location of the data 
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY OF THE TEST DATA OF DATA SET 3 in the XXXX space ####################

# Get the data using function "get_data_CL"
test_files, test_data, test_Ep, test_Eg, test_Eo, test_Br, test_Amp, test_Einf, test_BulkT, test_EMA_bool, test_EMAT, test_Sub_bool =  get_data_CL(path)

# Randomly Shuffle the data using function "unison_shuffled_copies"
test_files, test_data, test_Ep, test_Eg, test_Eo, test_Br, test_Amp, test_Einf, test_BulkT, test_EMA_bool, test_EMAT, test_Sub_bool  = unison_shuffled_copies( test_files, test_data, test_Ep, test_Eg, test_Eo, test_Br, test_Amp, test_Einf, test_BulkT, test_EMA_bool, test_EMAT, test_Sub_bool )

In [ ]:
idx = 5

print(test_files[idx])
print( test_Ep[idx])
print( test_Eg[idx])
print( test_Eo[idx])
print( test_Br[idx])
print( test_Amp[idx])
print( test_Einf[idx])
print( test_BulkT[idx])
print(test_EMA_bool[idx])
print(test_Sub_bool[idx])

In [ ]:
# Evaluate the performance of ANN1 on the test data. ANN1 is trained to predict the variable "test_Sub_bool".
# This variable is binary with "0" Si wafer substrate and "1" meaning SLG substrate

# Generate predictions from ANN1
ANN1_Predictions = ANN1.predict (test_data)

# Since the decision is binary, predictions < 0.5 will be rounded to 0 and predictions >= 0.5 will be rounded up to 1. 
num_correct = 0
num_wrong = 0

for i in range(len(test_data)): # iterate over training set

    if ANN1_Predictions[i][0] >= 0.5 and test_Sub_bool[i] >= 0.5: # If both prediction and ground truth are 1
        num_correct = num_correct + 1 # add one to correct count

    elif ANN1_Predictions[i][0] < 0.5 and test_Sub_bool[i] < 0.5: # If both prediction and ground truth are 0
        num_correct = num_correct + 1 # add one to correct count 

    else: # if prediction and ground truth do not match 
        num_wrong = num_wrong + 1 # add to wrong count

print( 'Fraction Correct: ' + str( num_correct / ( num_correct + num_wrong) ) ) # print fraction of correct predictions
print( 'Fraction Wrong: ' + str( num_wrong / ( num_correct + num_wrong) ) )  # print fraction of wrong predictions
    
    

ANN1 was able to predict every substrate in the test set correctly! 

In [ ]:
# Evaluate the performance of ANN2 on the test data. ANN2 is trained to predict the variable "test_EMA_bool".
# This variable is binary with "0" meaning no surface layer was used to simulate the data and "1" meaning a surface layer was used to simulate the data

# Generate predictions from ANN2
ANN2_Predictions = ANN2.predict(test_data)

# Since the decision is binary, predictions < 0.5 will be rounded to 0 and predictions >= 0.5 will be rounded up to 1. 
num_correct = 0
num_wrong = 0
num_wrong_index =[]
false_positive_SL_index = []
false_negative_SL_index = []

for i in range(len(test_data)): # iterate over training set

    if ANN2_Predictions[i][0] >= 0.5 and test_EMA_bool[i] >= 0.5: # If both prediction and ground truth are 1
        num_correct = num_correct + 1 # add one to correct count

    elif ANN2_Predictions[i][0] < 0.5 and test_EMA_bool[i] < 0.5: # If both prediction and ground truth are 0
        num_correct = num_correct + 1 # add one to correct count 

    elif ANN2_Predictions[i][0] > 0.5 and test_EMA_bool[i] < 0.5: # Flase positive
        num_wrong = num_wrong + 1 # add to wrong count
        num_wrong_index.append(i) # stores the index of any wrong predictions
        false_positive_SL_index.append(i)

    elif ANN2_Predictions[i][0] < 0.5 and test_EMA_bool[i] > 0.5: # Flase negative
        num_wrong = num_wrong + 1 # add to wrong count
        num_wrong_index.append(i) # stores the index of any wrong predictions
        false_negative_SL_index.append(i)


print( 'Fraction Correct: ' + str( num_correct / ( num_correct + num_wrong) ) )
print( 'Fraction Wrong: ' + str( num_wrong / ( num_correct + num_wrong) ) ) 
print( 'Number Correct: ' + str( num_correct )  )
print( 'Number Wrong: ' + str( num_wrong )  )
print( 'Number of False Positive: ' + str( len(false_positive_SL_index)  ))
print( 'Number of False Negative: ' + str( len(false_negative_SL_index)  ))    

The ANN2 was correct nearly 90% of the time and ANN1 was correct 100% of the time. Now these ANNs will be provided the experimental data. 
Now the trials that were predicted incorrectly will be examined.

The substrate, presence of surface layer, and surface layer thickness will be looked at.

In [ ]:
Si_sub = 0  # number of failed trials with Si substrate 
SL = 0 # number of failed trials with a surface layer
SL_small = 0 # number of failed trials where the surface layer thickness is < 0.5 nm
bulk_small = 0 # number of failed trials where bulk thickness < 75 nm 
bulk_avg = 0 # bulk AVG for failed trials
Eo_avg = 0
Eg_avg = 0
Ep_avg = 0
Br_avg = 0
Amp_avg = 0


for i in range(len(num_wrong_index)): # iterate for every failed trial

    if test_Sub_bool[num_wrong_index[i]] < 0.5: # if substrate is Si wafer
        Si_sub = Si_sub + 1 # add 1 to Si_sub

    if test_EMA_bool[num_wrong_index[i]] > 0.5: # if there is a surface layer
        SL = SL + 1 # add 1 to SL 

        if test_EMAT[num_wrong_index[i]] <= 0.5: # if surface layer thickness is less than 0.5 nm 
            SL_small = SL_small + 1 # add 1 to SL_small

    if test_BulkT[num_wrong_index[i]] <= 75: 
        bulk_small = bulk_small + 1
    
    bulk_avg = bulk_avg + test_BulkT[num_wrong_index[i]]
    Eo_avg = Eo_avg + test_Eo[num_wrong_index[i]]
    Eg_avg = Eg_avg + test_Eg[num_wrong_index[i]]
    Ep_avg = Ep_avg + test_Ep[num_wrong_index[i]]
    Br_avg = Br_avg + test_Br[num_wrong_index[i]]
    Amp_avg = Amp_avg + test_Amp[num_wrong_index[i]]
    #print(test_Amp[num_wrong_index[i]])

bulk_avg = bulk_avg / len(num_wrong_index)
Eo_avg = Eo_avg / len(num_wrong_index)
Eg_avg = Eg_avg / len(num_wrong_index)
Ep_avg = Ep_avg / len(num_wrong_index)
Br_avg = Br_avg / len(num_wrong_index)
Amp_avg = Amp_avg / len(num_wrong_index)

print('Number of failures with Si Substrate: ' + str( Si_sub ) )
print('Number of failures with Surface Layer: ' + str( SL ) )
print('Number of failures with Surface Layer less than 0.5 nm: ' + str( SL_small ) )
print('Number of failures with Bulk less than 75 nm: ' + str( bulk_small ) )
print('Bulk Avereage: ' + str( bulk_avg) )
print('Eo Avereage: ' + str( Eo_avg) )
print('Eg Avereage: ' + str( Eg_avg) )
print('Ep Avereage: ' + str( Ep_avg) )
print('Br Avereage: ' + str( Br_avg) )
print('Amp Avereage: ' + str( Amp_avg) )

In [ ]:
# Examine the trials that failed. Starting with the substrate

data = test_Sub_bool[num_wrong_index] # det subset of data from failed trials 

# Define the range and number of bins for histogram
min_value = 0
max_value = 1
num_bins = 2


# Create bins using np.linspace to generate equally spaced bins
bins = np.linspace(min_value, max_value, num_bins + 1)

# Count the number of data points in each bin
hist, edges = np.histogram(data, bins)

# Print the counts for each bin
for count, edge in zip(hist, edges[:-1]):
    print(f'Bin {edge:.2f} to {edges[edges.tolist().index(edge) + 1]:.2f}: {count}')

labels = ['c-Si Substrate', 'SLG Substrate']

# Optional: Plot the histogram
plt.hist(data, bins, edgecolor='black')
plt.xticks(edges)
plt.xticks([0.25, 0.75], labels=labels )
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.show()


In [ ]:
# Examine the trials that failed. Starting with the substrate

data = test_EMA_bool[num_wrong_index] # det subset of data from failed trials 

# Define the range and number of bins for histogram
min_value = 0
max_value = 1
num_bins = 2


# Create bins using np.linspace to generate equally spaced bins
bins = np.linspace(min_value, max_value, num_bins + 1)

# Count the number of data points in each bin
hist, edges = np.histogram(data, bins)

# Print the counts for each bin
for count, edge in zip(hist, edges[:-1]):
    print(f'Bin {edge:.2f} to {edges[edges.tolist().index(edge) + 1]:.2f}: {count}')

labels = ['No Surface Layer', 'Surface Layer']

# Optional: Plot the histogram
plt.hist(data, bins, edgecolor='black')
plt.xticks([0.25, 0.75], labels=labels )
plt.tick_params(axis='both', direction='in', length=0)
#plt.xlabel('Value')
plt.ylabel('Frequency')
plt.show()


In [ ]:
# Examine the trials that failed. Starting with the substrate

data = test_EMAT[num_wrong_index] # det subset of data from failed trials 

# Define the range and number of bins for histogram
min_value = 0.1
max_value = 2
num_bins = 10


# Create bins using np.linspace to generate equally spaced bins
bins = np.linspace(min_value, max_value, num_bins + 1)

# Count the number of data points in each bin
hist, edges = np.histogram(data, bins)

# Print the counts for each bin
for count, edge in zip(hist, edges[:-1]):
    print(f'Bin {edge:.2f} to {edges[edges.tolist().index(edge) + 1]:.2f}: {count}')


# Optional: Plot the histogram
plt.hist(data, bins, edgecolor='black')
plt.xticks(edges)
plt.tick_params(axis='both', direction='in', length=6)
plt.xlabel('Surface Layer Thickness (nm)')
plt.ylabel('Frequency')
#plt.title('Histogram of Surface Layer Thickness Failures')
plt.show()

In [ ]:
# Examine the trials that failed. Starting with the substrate

data = test_BulkT[false_negative_SL_index] # det subset of data from failed trials 

# Define the range and number of bins for histogram
min_value = 25
max_value = 125
num_bins = 10


# Create bins using np.linspace to generate equally spaced bins
bins = np.linspace(min_value, max_value, num_bins + 1)

# Count the number of data points in each bin
hist, edges = np.histogram(data, bins)

# Print the counts for each bin
for count, edge in zip(hist, edges[:-1]):
    print(f'Bin {edge:.2f} to {edges[edges.tolist().index(edge) + 1]:.2f}: {count}')


# Optional: Plot the histogram
plt.hist(data, bins, edgecolor='black')
plt.xticks(edges)
plt.tick_params(axis='both', direction='in', length=6)
plt.xlabel('Bulk Thickness (nm)')
plt.ylabel('Frequency')
plt.show()

According to the traditional Least-square regression analysis, samples MV1519 and MV1523 are not sensitive to a surface layer in the analysis, but the MV1530 sample is sensitive to a surface layer. Also, samples MV1519 and MV1523 are fabricated on Si wafer while sample MV1530 is fabricated on SLG.

In [ ]:
# Evaluate the performance of the ANN1 on the experimental data. .
# This variable is binary with "0" meaning Si wafer  needed "1" meaning SLG for the substrate
# MV1519 and MV1523 both have Si wafer substrate, but MV1530 has an SLG substrate

# Generate predictions from ANN1 on experimental data 
ANN1_Predictions_MV1519 = ANN1.predict(MV1519)
ANN1_Predictions_MV1523 = ANN1.predict(MV1523)
ANN1_Predictions_MV1530 = ANN1.predict(MV1530)

# score values for MV1519
num_correct_MV1519 = 0
num_wrong_MV1519 = 0

# score values for MV1523
num_correct_MV1523 = 0
num_wrong_MV1523 = 0

# score values for MV1530
num_correct_MV1530 = 0
num_wrong_MV1530 = 0

for i in range(81): # iterate over each 81 point map

    # Check values for MV1519 
    if ANN1_Predictions_MV1519[i][0] < 0.5: # we know MV1519 value should always be '0' according the LSR
        num_correct_MV1519 = num_correct_MV1519 + 1

    else: num_wrong_MV1519 = num_wrong_MV1519 + 1

    # Check values for MV1523
    if ANN1_Predictions_MV1523[i][0] < 0.5: # we know MV1523 value should always be '0' according the LSR
        num_correct_MV1523 = num_correct_MV1523 + 1

    else: num_wrong_MV1523 = num_wrong_MV1523 + 1

    # Check values for MV1530
    if ANN1_Predictions_MV1530[i][0] >= 0.5: # we know MV1530 value should always be '1' according the LSR
        num_correct_MV1530 = num_correct_MV1530 + 1

    else: num_wrong_MV1530 = num_wrong_MV1530 + 1



print( 'Fraction Correct MV1519: ' + str( num_correct_MV1519 / ( num_correct_MV1519 + num_wrong_MV1519 ) ) )
print( 'Fraction Wrong MV1519: ' + str( num_wrong_MV1519  / ( num_correct_MV1519  + num_wrong_MV1519 ) ) ) 
print( 'Fraction Correct MV1523: ' + str( num_correct_MV1523 / ( num_correct_MV1523 + num_wrong_MV1523 ) ) )
print( 'Fraction Wrong MV1523: ' + str( num_wrong_MV1523  / ( num_correct_MV1523  + num_wrong_MV1523 ) ) ) 
print( 'Fraction Correct MV1530: ' + str( num_correct_MV1530 / ( num_correct_MV1530 + num_wrong_MV1530 ) ) )
print( 'Fraction Wrong MV1530: ' + str( num_wrong_MV1530  / ( num_correct_MV1530  + num_wrong_MV1530 ) ) ) 
    
    

In [ ]:
# Evaluate the performance of the ANN2 on the experimental data. .
# This variable is binary with "0" meaning no surface layer is needed "1" meaning a surface layer is needed
# MV1519 and MV1523 both do not have surface layers in traditional analysis, but MV1530 needs a surface layer

# Generate predictions from Surface_Layer_ANN on experimental data 
ANN2_Predictions_MV1519 = ANN2.predict(MV1519)
ANN2_Predictions_MV1523 = ANN2.predict(MV1523)
ANN2_Predictions_MV1530 = ANN2.predict(MV1530)

# score values for MV1519
num_correct_MV1519 = 0
num_wrong_MV1519 = 0

# score values for MV1523
num_correct_MV1523 = 0
num_wrong_MV1523 = 0

# score values for MV1530
num_correct_MV1530 = 0
num_wrong_MV1530 = 0

for i in range(81): # iterate over each 81 point map

    # Check values for MV1519 
    if ANN2_Predictions_MV1519[i][0] < 0.5: # we know MV1519 value should always be '0' according the LSR
        num_correct_MV1519 = num_correct_MV1519 + 1

    else: num_wrong_MV1519 = num_wrong_MV1519 + 1

    # Check values for MV1523
    if ANN2_Predictions_MV1523[i][0] < 0.5: # we know MV1523 value should always be '0' according the LSR
        num_correct_MV1523 = num_correct_MV1523 + 1

    else: num_wrong_MV1523 = num_wrong_MV1523 + 1

    # Check values for MV1530
    if ANN2_Predictions_MV1530[i][0] >= 0.5: # we know MV1530 value should always be '1' according the LSR
        num_correct_MV1530 = num_correct_MV1530 + 1

    else: num_wrong_MV1530 = num_wrong_MV1530 + 1



print( 'Fraction Correct MV1519: ' + str( num_correct_MV1519 / ( num_correct_MV1519 + num_wrong_MV1519 ) ) )
print( 'Fraction Wrong MV1519: ' + str( num_wrong_MV1519  / ( num_correct_MV1519  + num_wrong_MV1519 ) ) ) 
print( 'Fraction Correct MV1523: ' + str( num_correct_MV1523 / ( num_correct_MV1523 + num_wrong_MV1523 ) ) )
print( 'Fraction Wrong MV1523: ' + str( num_wrong_MV1523  / ( num_correct_MV1523  + num_wrong_MV1523 ) ) ) 
print( 'Fraction Correct MV1530: ' + str( num_correct_MV1530 / ( num_correct_MV1530 + num_wrong_MV1530 ) ) )
print( 'Fraction Wrong MV1530: ' + str( num_wrong_MV1530  / ( num_correct_MV1530  + num_wrong_MV1530 ) ) ) 
    
    

ANN1 and ANN2 are in complete agreement with the traditional least-square regression analysis! Now evaluate the performance of the ANN3 and ANN4  

In [ ]:
# load in mean and standard deviation for each size training set

# change directory to location of files 
os.chdir(r'XXXX')  ########## PLEASE PUT THE DIRECTORY WHERE YOU STORED THE MEAN AND STANDARD DEVIATIONS FOR EACH DATA SET ####################
############ THE MEAN AND STANDARD DEVIATIONS SHOULD HAVE BEEN MADE IN THE "Model_Development" WORKBOOK ########################

file = 'Data_Set1_10_Mean.csv' # mean for data set of size 10
Data_Set1_10_Mean = pd.read_csv(file) # read in files and pandas dataframe

file = 'Data_Set1_10_std.csv'  # mean standard deviation for data set of size 10
Data_Set1_10_std = pd.read_csv(file) # read in files and pandas dataframe

file = 'Data_Set1_100_Mean.csv' # mean for data set of size 100
Data_Set1_100_Mean = pd.read_csv(file) # read in files and pandas dataframe

file = 'Data_Set1_100_std.csv'  # mean standard deviation for data set of size 100
Data_Set1_100_std = pd.read_csv(file) # read in files and pandas dataframe

file = 'Data_Set1_1000_Mean.csv' # mean for data set of size 1000
Data_Set1_1000_Mean = pd.read_csv(file) # read in files and pandas dataframe

file = 'Data_Set1_1000_std.csv'  # mean standard deviation for data set of size 1000
Data_Set1_1000_std = pd.read_csv(file) # read in files and pandas dataframe

file = 'Data_Set1_10000_Mean.csv' # mean for data set of size 10000
Data_Set1_10000_Mean = pd.read_csv(file) # read in files and pandas dataframe

file = 'Data_Set1_10000_std.csv'  # mean standard deviation for data set of size 10000
Data_Set1_10000_std = pd.read_csv(file) # read in files and pandas dataframe

file = 'Data_Set1_50000_Mean.csv' # mean for data set of size 50000
Data_Set1_50000_Mean = pd.read_csv(file) # read in files and pandas dataframe

file = 'Data_Set1_50000_std.csv'  # mean standard deviation for data set of size 50000
Data_Set1_50000_std = pd.read_csv(file) # read in files and pandas dataframe

file = 'Data_Set1_100000_Mean.csv' # mean for data set of size 50000
Data_Set1_100000_Mean = pd.read_csv(file) # read in files and pandas dataframe

file = 'Data_Set1_100000_std.csv'  # mean standard deviation for data set of size 50000
Data_Set1_100000_std = pd.read_csv(file) # read in files and pandas dataframe


In [ ]:
# Load in all the ANNs 

# move to location of the pre-trained models
os.chdir(r"XXXX") ########## PLEASE PUT THE DIRECTORY TO THE FOLDER LABELED "Models" in the XXXX space ####################

# Load in models for the different versions of ANN3
ANN3_10 = load_model('ANN3_10.keras')
ANN3_100 = load_model('ANN3_100.keras')
ANN3_1000 = load_model('ANN3_1000.keras')
ANN3_10000 = load_model('ANN3_10000.keras')
ANN3_50000 = load_model('ANN3_50000.keras')
ANN3_100000 = load_model('ANN3_100000.keras')



In [ ]:
# Load in the 10000 test data set.

# Identify the location of the data 
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY OF THE TEST DATA OF DATA SET 1 in the XXXX space ####################

# Get the data using function "get_data_CL"
test_files, test_data, test_Ep, test_Eg, test_Eo, test_Br, test_Amp, test_Einf, test_BulkT, test_EMA_bool, test_EMAT, test_Sub_bool =  get_data_CL(path)

# Randomly Shuffle the data using function "unison_shuffled_copies"
test_files, test_data, test_Ep, test_Eg, test_Eo, test_Br, test_Amp, test_Einf, test_BulkT, test_EMA_bool, test_EMAT, test_Sub_bool  = unison_shuffled_copies( test_files, test_data, test_Ep, test_Eg, test_Eo, test_Br, test_Amp, test_Einf, test_BulkT, test_EMA_bool, test_EMAT, test_Sub_bool )


# Save test parameters for later analysis
y_test_DS1 = {

    'Ep' :  test_Ep,
    'Eg' :  test_Eg,
    'Eo' :  test_Eo,
    'Br' :  test_Br,
    'Amp' : test_Amp,
    'BulkT': test_BulkT,

}

In [ ]:
# Generate predictions on the test data 

ANN3_10_test = ANN3_10.predict(test_data)
ANN3_100_test = ANN3_100.predict(test_data)
ANN3_1000_test = ANN3_1000.predict(test_data)
ANN3_10000_test = ANN3_10000.predict(test_data)
ANN3_50000_test = ANN3_50000.predict(test_data)
ANN3_100000_test = ANN3_100000.predict(test_data)

A function is defined to unstandardize the data.

In [ ]:
#Define a function to help un-standardize the data once it is generated by the ANN
def UnStandardize_ANN_Output(Standard_Means, Standard_Std, Model_Predictions): 
    """
    Standard_Means: A pandas dataframe containing all the mean values
    Standard_Std: A pandas dataframe containing all the standard deviation values
    Model_Predictions: The output of an ANN model we want to unstandardize
    """
    
    Model_Predicted_Ep = (Model_Predictions[:,0] * Standard_Std['std_Ep'][0]) + Standard_Means['Mean_Ep'][0]
    Model_Predicted_Eg = (Model_Predictions[:,1] * Standard_Std['std_Eg'][0]) + Standard_Means['Mean_Eg'][0]
    Model_Predicted_Eo = (Model_Predictions[:,2] * Standard_Std['std_Eo'][0]) + Standard_Means['Mean_Eo'][0]
    Model_Predicted_Br = (Model_Predictions[:,3] * Standard_Std['std_Br'][0]) + Standard_Means['Mean_Br'][0]
    Model_Predicted_Amp = (Model_Predictions[:,4] * Standard_Std['std_Amp'][0]) + Standard_Means['Mean_Amp'][0]
    Model_Predicted_BulkT = (Model_Predictions[:,5] * Standard_Std['std_BulkT'][0]) + Standard_Means['Mean_BulkT'][0]

    
    if len( Model_Predictions[0]) == 7: # If there is an EMA layer included
        
        Model_Predicted_EMAT = (Model_Predictions[:,6] * Standard_Std['std_EMAT'][0]) + Standard_Means['Mean_EMAT'][0]
        
        dict1 = {
        'Ep' : Model_Predicted_Ep,
        'Eg' :  Model_Predicted_Eg, 
        'Eo' :  Model_Predicted_Eo,
        'Br' :  Model_Predicted_Br, 
        'Amp':  Model_Predicted_Amp,
        'BulkT': Model_Predicted_BulkT,
        'EMAT': Model_Predicted_EMAT 
        }
    
        return(dict1)

    if len(Model_Predictions[0]) == 6: # If there is NO EMA layer included
        
        dict1 = {
        'Ep' :  Model_Predicted_Ep,
        'Eg' :  Model_Predicted_Eg, 
        'Eo' :  Model_Predicted_Eo,
        'Br' :  Model_Predicted_Br, 
        'Amp':  Model_Predicted_Amp,
        'BulkT': Model_Predicted_BulkT 
        }
        return(dict1)
        


In [ ]:
# Unstandardize the data 

ANN3_10_test = UnStandardize_ANN_Output( Data_Set1_10_Mean, Data_Set1_10_std, ANN3_10_test)
ANN3_100_test = UnStandardize_ANN_Output( Data_Set1_100_Mean, Data_Set1_100_std, ANN3_100_test)
ANN3_1000_test = UnStandardize_ANN_Output( Data_Set1_1000_Mean, Data_Set1_1000_std, ANN3_1000_test)
ANN3_10000_test = UnStandardize_ANN_Output( Data_Set1_10000_Mean, Data_Set1_10000_std, ANN3_10000_test)
ANN3_50000_test = UnStandardize_ANN_Output( Data_Set1_50000_Mean, Data_Set1_50000_std, ANN3_50000_test)
ANN3_100000_test = UnStandardize_ANN_Output( Data_Set1_100000_Mean, Data_Set1_100000_std, ANN3_100000_test)



In [ ]:
# Now generate predictions on the experimental data 

ANN3_10_MV1519 = ANN3_10.predict(MV1519)
ANN3_10_MV1523 = ANN3_10.predict(MV1523)

ANN3_100_MV1519 = ANN3_100.predict(MV1519)
ANN3_100_MV1523 = ANN3_100.predict(MV1523)

ANN3_1000_MV1519 = ANN3_1000.predict(MV1519)
ANN3_1000_MV1523 = ANN3_1000.predict(MV1523)

ANN3_10000_MV1519 = ANN3_10000.predict(MV1519)
ANN3_10000_MV1523 = ANN3_10000.predict(MV1523)

ANN3_50000_MV1519 = ANN3_50000.predict(MV1519)
ANN3_50000_MV1523 = ANN3_50000.predict(MV1523)

ANN3_100000_MV1519 = ANN3_100000.predict(MV1519)
ANN3_100000_MV1523 = ANN3_100000.predict(MV1523)

In [ ]:
# Now unstandardize the predictions

ANN3_10_MV1519 = UnStandardize_ANN_Output( Data_Set1_10_Mean, Data_Set1_10_std, ANN3_10_MV1519)
ANN3_10_MV1523 = UnStandardize_ANN_Output( Data_Set1_10_Mean, Data_Set1_10_std, ANN3_10_MV1523)

ANN3_100_MV1519 = UnStandardize_ANN_Output( Data_Set1_100_Mean, Data_Set1_100_std, ANN3_100_MV1519)
ANN3_100_MV1523 = UnStandardize_ANN_Output( Data_Set1_100_Mean, Data_Set1_100_std, ANN3_100_MV1523)

ANN3_1000_MV1519 = UnStandardize_ANN_Output( Data_Set1_1000_Mean, Data_Set1_1000_std, ANN3_1000_MV1519)
ANN3_1000_MV1523 = UnStandardize_ANN_Output( Data_Set1_1000_Mean, Data_Set1_1000_std, ANN3_1000_MV1523)

ANN3_10000_MV1519 = UnStandardize_ANN_Output( Data_Set1_10000_Mean, Data_Set1_10000_std, ANN3_10000_MV1519)
ANN3_10000_MV1523 = UnStandardize_ANN_Output( Data_Set1_10000_Mean, Data_Set1_10000_std, ANN3_10000_MV1523)

ANN3_50000_MV1519 = UnStandardize_ANN_Output( Data_Set1_50000_Mean, Data_Set1_50000_std, ANN3_50000_MV1519)
ANN3_50000_MV1523 = UnStandardize_ANN_Output( Data_Set1_50000_Mean, Data_Set1_50000_std, ANN3_50000_MV1523)

ANN3_100000_MV1519 = UnStandardize_ANN_Output( Data_Set1_100000_Mean, Data_Set1_100000_std, ANN3_100000_MV1519)
ANN3_100000_MV1523 = UnStandardize_ANN_Output( Data_Set1_100000_Mean, Data_Set1_100000_std, ANN3_100000_MV1523)


# Now store the predictions
os.chdir(r'XXXX')  ########## PLEASE PUT THE DIRECTORY FOR WHERE YOU WOULD LIKE THE ANN PREDICTIONS TO BE STORED ####################

df = pd.DataFrame(ANN3_10_MV1519)
df.to_csv('ANN3_10_MV1519.csv')

df = pd.DataFrame(ANN3_100_MV1519)
df.to_csv('ANN3_100_MV1519.csv')

df = pd.DataFrame(ANN3_1000_MV1519)
df.to_csv('ANN3_1000_MV1519.csv')

df = pd.DataFrame(ANN3_10000_MV1519)
df.to_csv('ANN3_10000_MV1519.csv')

df = pd.DataFrame(ANN3_50000_MV1519)
df.to_csv('ANN3_50000_MV1519.csv')

df = pd.DataFrame(ANN3_100000_MV1519)
df.to_csv('ANN3_100000_MV1519.csv')

df = pd.DataFrame(ANN3_10_MV1523)
df.to_csv('ANN3_10_MV1523.csv')

df = pd.DataFrame(ANN3_100_MV1523)
df.to_csv('ANN3_100_MV1523.csv')

df = pd.DataFrame(ANN3_1000_MV1523)
df.to_csv('ANN3_1000_MV1523.csv')

df = pd.DataFrame(ANN3_10000_MV1523)
df.to_csv('ANN3_10000_MV1523.csv')

df = pd.DataFrame(ANN3_50000_MV1523)
df.to_csv('ANN3_50000_MV1523.csv')

df = pd.DataFrame(ANN3_100000_MV1523)
df.to_csv('ANN3_100000_MV1523.csv')


Now the predictions ANN3 are ready. Ellipsometric spectra will be generated for each prediction and compared to the experimental data by calculating the unweighted error function between ANN and experiment. The traditional least-square regression (LSR) unweighted error function will also be calculated. 

In [ ]:
# Now generate the NCS spectra for the traditionaLSR analysis 

############## MV1519 LSR ###########################################
LSR_MV1519_NCS =[]
LSR_MV1519_CL_List =[]
for i in range(len(MV1519_params['Amp'])):
    
    #first get Cody-Lorentz oscillator parameters
    E_inf = 1 # range fixed to 1
    Ep  = float(MV1519_params['Ep'][i])
    Amp  = float(MV1519_params['Amp'][i])
    Br =   float(MV1519_params['Br'][i])
    Eo =   float(MV1519_params['Eo'][i])
    Eg =   float(MV1519_params['Eg'][i])
    Et = 0 #this term being 0 effectivly neglects the Urbach energy and simplifies the equations.
    Egt = Eg + Et
    
    # Now we generate our Cody-Lorentz material
    CL = Get_CL_Material(E, Ep, Eg, Eo, Br, Amp, Egt, E_inf, wv)
    LSR_MV1519_CL_List.append(CL)
    #Generate Thicknesses
    Bulk_Thickness = float(MV1519_params['BulkT'][i])
    NTVE_JAW_Thickness = 1.68
    
    #define Structure 
    Structure = [Void, CL, NTVE_JAW, Si_JAW]
    #Define material thicknesses
    Mat_Thick = np.array( [Bulk_Thickness, NTVE_JAW_Thickness] )
    #Define angle of incidence
    Theta_Incident = 64.93
    df = SE_Sim(Structure, Theta_Incident,  Mat_Thick, write_data=False, NCS=True)
    df = df.drop(df.columns[0], axis=1)
    LSR_MV1519_NCS.append(df)

############## MV1523 LSR ###########################################

LSR_MV1523_NCS =[]
LSR_MV1523_CL_List =[]
for i in range(len(MV1523_params['Amp'])):
    
    #first get Cody-Lorentz Oscillator parameters
    E_inf = 1 # range fixed to 1
    Ep  = float(MV1523_params['Ep'][i])
    Amp  = float(MV1523_params['Amp'][i])
    Br =   float(MV1523_params['Br'][i])
    Eo =   float(MV1523_params['Eo'][i])
    Eg =   float(MV1523_params['Eg'][i])
    Et = 0 #this term being 0 effectivly neglects the Urbach energy and simplifies the equations.
    Egt = Eg + Et
    
    # Now we generate our Cody-Lorentz material
    CL = Get_CL_Material(E, Ep, Eg, Eo, Br, Amp, Egt, E_inf, wv)
    LSR_MV1523_CL_List.append(CL)
    #Generate Thicknesses
    Bulk_Thickness = float(MV1523_params['BulkT'][i])
    NTVE_JAW_Thickness = 1.65
    
    #define Structure 
    Structure = [Void, CL, NTVE_JAW, Si_JAW]
    #Define material thicknesses
    Mat_Thick = np.array( [Bulk_Thickness, NTVE_JAW_Thickness] )
    #Define angle of incidence
    Theta_Incident = 64.93
    df = SE_Sim(Structure, Theta_Incident,  Mat_Thick, write_data=False, NCS=True)
    df = df.drop(df.columns[0], axis=1)
    LSR_MV1523_NCS.append(df)

Define a function that can calculate the ellipsometric spectra for a given set of ANN predictions

In [ ]:
def NCS_Genorator_ANN3(ANN_List, NTVE_JAW_Thickness, E, wv): 

    """
    The goal of this function is to generate ellipsometric spectra for each ANN output in the ANN_List. 
    ANN_List should be a list of dictionaries. Each dictionary contains a set of predictions for a given ANN model
    """
    NCS_List = [] #stores the ellipsometric specta
    CL_List = []
    for j in range(len(ANN_List)): # iterate over the entire list of ANN output
        print(j) # just to make sure function is working
        modX_NCS_List = [] # list containing values for a specific model
        modX_CL_List = [] # list containing values for a specific model
            
        for i in range(len(ANN_List[j]['Eg'])): #iterate over each prediction for a given ANN
    
            #first get Cody-Loretnz Oscillator parameters
            E_inf = 1 # range fixed to 1
            Ep  = ANN_List[j]['Ep'][i]
            Amp  = ANN_List[j]['Amp'][i]
            Br =   ANN_List[j]['Br'][i]
            Eo =   ANN_List[j]['Eo'][i]
            Eg =   ANN_List[j]['Eg'][i]
            Et = 0 #this term being 0 effectivly neglects the Urbach energy and simplifies the equations.
            Egt = Eg + Et
        
            # Now we generate our Cody-Lorentz material
            CL = Get_CL_Material(E, Ep, Eg, Eo, Br, Amp, Egt, E_inf, wv)
            modX_CL_List.append(CL)
            #Now we define our structure

            Bulk_Thickness = ANN_List[j]['BulkT'][i]
            #EMA_Thickness =  mod5_SE_predicted_EMAT[i][0]
              
        
            #define Structure 
            Structure = [Void,  CL, NTVE_JAW, Si_JAW]
            Mat_Thick =  np.array( [ Bulk_Thickness, NTVE_JAW_Thickness ] )
            Theta_Incident = 64.93
            df = SE_Sim(Structure, Theta_Incident,  Mat_Thick, write_data=False, NCS=True)
            df = df.drop(df.columns[0], axis=1)
            modX_NCS_List.append(df) # store prediction in the list for that model
            
        
        NCS_List.append(modX_NCS_List) # store that entire model's predictions as 1 term in this list
        CL_List.append(modX_CL_List)
    return(NCS_List, CL_List)
        
                  

In [ ]:
# Put outputs in lists 
MV1519_list = [ANN3_10_MV1519, ANN3_100_MV1519, ANN3_1000_MV1519, ANN3_10000_MV1519, ANN3_50000_MV1519, ANN3_100000_MV1519]
MV1523_list = [ANN3_10_MV1523, ANN3_100_MV1523, ANN3_1000_MV1523, ANN3_10000_MV1523, ANN3_50000_MV1523, ANN3_100000_MV1523]

# Calculate ellipsometric spectra
ANN3_MV1519_NCS, ANN3_MV1519_CL = NCS_Genorator_ANN3(MV1519_list, 1.68, E, wv) # native oxide thickness treated as fixed 1.68nm
ANN3_MV1523_NCS, ANN3_MV1523_CL = NCS_Genorator_ANN3(MV1523_list, 1.65, E, wv) # native oxide thickness treated as fixed 1.65nm

Now we can compare these spectra to the experimental data and the least-square regression. The unweighted error function will be generate for each point to help make comparisons. The average error will be desplayed. A function that makes these comparisons is defined to compare the data from ANN3. This function will also export figures containing this data.

In [ ]:
# First define the unweighted error function

def SE_Sigma(array, df, m):
    """
    Calculate the unweighted error function between the measured ellipsometric spectra and the simulated spectra
    
    Parameters:
    - array: NumPy array - Measured ellipsometric spectra
    - df: Pandas DataFrame - Simulated ellipsometric spectra
    - m: number of fit parameters for optical and structural model
    
    Returns:
    - Sigma: The unweighted error function
    """
    # Ensure the array has the same shape as the DataFrame
    if array.shape != df.shape:
        raise ValueError("NumPy array and DataFrame must have the same shape")
    
    # Convert the NumPy array to a DataFrame for easier handling
    n = array.shape[0] * array.shape[1]
    array_df = pd.DataFrame(array, columns=df.columns, index=df.index) 
    array_df.columns = ['N', 'C', 'S']
    
    # Calculate Sigma

    #Calculate (N - N')^2  we will call this variable N_diff
    N_diff = ( (df['N'] - array_df['N'])**2 ).sum()

    #Calculate (C - C')^2  we will call this variable C_diff
    C_diff = ( (df['C'] - array_df['C'])**2 ).sum()

    #Calculate (S - S')^2  we will call this variable S_diff
    S_diff = ( (df['S'] - array_df['S'])**2 ).sum()

    #Now we calculate n
    n = array.shape[0] # should be 697 for the intended spectral range

    #Now Put everything together to get Sigma

    Sigma = ( (1/(3*n - m)) * (N_diff + C_diff + S_diff ) )**(1/2)

    return Sigma    


In [ ]:
def NCS_Comparison_ANN3(ANN_NCS_List, Experimental_NCS, Least_Square_NCS, param_num, path, generate_images=False, foldername = ''):

    """
    Goal of function is to iterate over every ANN NCS list and compare each ANN list to the corresponsinf experimental 
    and least-square regression analysis. 

    If generate_images is set to True, then pictures for wach spectra will be generated.
    """

    #iterate over the ANN_NCS_list

    Sigma_mod_avg_list = [] # list to store the average value of sigma for each model
    Sigma_mod_grand_list =[] # list to store every value for sigma for each model

    Sigma_lsq_avg_list =[]
    Sigma_lsq_grand_list = []

    for k in range(len(ANN_NCS_List)): #iterate for each ANN model

        #create path to store images if  generate_images=True
        folder = str(  'ANN3_' + str(10**(k + 1)) + '_' + foldername)
        if k == 4: 
            folder = 'ANN3_' + str(50000) +  '_' + foldername
        if k == 5: 
            folder = 'ANN3_' + str(100000)  + '_' + foldername
        
        if generate_images:
            os.makedirs(path + folder, exist_ok=True)
            os.chdir(path + folder)

        #initalize values
        Sigma_mod_list =[]
        Sigma_mod_avg = 0 #stores the average value of Sigma 
        
        Sigma_LeastSquare_avg = 0 # stores the average value of Sigma for least square regression 
        Sigma_LeastSquare_list = []

        for i in range(len(ANN_NCS_List[k])):# iterate for each measurement. In this case, there are 81 measurements 

            #Calcualte Sigma between ANN prediction N,C,S spectra and real SE data
            #print('Exp:', Experimental_NCS[i].shape)
            #print('ANN:', ANN_NCS_List[k][i].shape)
            Sigma_mod = SE_Sigma(Experimental_NCS[i], ANN_NCS_List[k][i], param_num)
            Sigma_mod_avg = Sigma_mod_avg + Sigma_mod
            Sigma_mod_list.append(Sigma_mod)
            
            #plt.text(5, 1, 'ANN Model σ: ' + str(round(Sigma_mod, 5)) , fontsize=10, ha='center')
            
            
            #Calcualte MSE between LSR N,C,S spectra and real SE data
            #print('Exp:', Experimental_NCS[i].shape)
            #print('LSR:', Least_Square_NCS[i].shape)
            Sigma_LSQ = SE_Sigma(Experimental_NCS[i], Least_Square_NCS[i], param_num)
            Sigma_LeastSquare_avg = Sigma_LeastSquare_avg + Sigma_LSQ
            Sigma_LeastSquare_list.append(Sigma_LSQ)

            def format_ticks(x, _):
                return f'{x * 1000:.0f}'  # Multiply by scale_factor

            
            # Create a figure with 3 stacked subplots
            fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(10, 5)) 

            axs = axs.flatten()

            # Plot N for experiment, ANN, and LSR
            axs[0].scatter(E,  Experimental_NCS[i][:, 0], facecolor='none', edgecolor = 'blue', s =10,   label='Experiment'  )
            axs[0].plot(E,  Least_Square_NCS[i]['N'], color='black', linestyle='--', label='LSR', linewidth=3 )
            axs[0].plot(E,  ANN_NCS_List[k][i]['N'], color='red', linestyle='--', label='ANN', linewidth=3 )
            axs[0].set_xlim(0.7, 5.8)
            axs[0].set_xticks(np.arange(1, 5.5, 1))

            axs[0].set_ylim(0.2, 1)
            axs[0].set_yticks(np.arange(0.3, 0.95, 0.3))
            
            axs[0].set_ylabel('$N$= $Cos2$' + r'$\Psi$ ', fontsize=10, labelpad=9)  # Title for y-axis
            axs[0].text(4.15, 0.85, 'ANN σ: ' + str("{:.4f}".format(Sigma_mod)), fontsize =10)
            axs[0].text(4.15, 0.75, 'LSR  σ: ' + str("{:.4f}".format(Sigma_LSQ)), fontsize =10 )
            axs[0].tick_params(axis='both', which='major', labelsize=10)  # Adjust labelsize as needed
            axs[0].tick_params(axis='x', which='both', top=True, bottom=True)  # Ticks on top and bottom
            axs[0].tick_params(axis='y', which='both', left=True, right=True)  # Ticks on left and right

   
            # Plot N for experiment, ANN, and LSR
            #axs[1].scatter(E,  Experimental_NCS[i][:, 0], facecolor='none', edgecolor = 'blue', s =10,   label='Experiment'  )
            axs[1].plot(E,  Experimental_NCS[i][:, 0] - Least_Square_NCS[i]['N'], color='black', linestyle='--', label='LSR', linewidth=3 )
            axs[1].plot(E, Experimental_NCS[i][:, 0] - ANN_NCS_List[k][i]['N'], color='red', linestyle='--', label='ANN', linewidth=3 )
            axs[1].set_xlim(0.7, 5.8)
            axs[1].set_ylim(-0.018, 0.018)
            axs[1].set_ylabel(r"$(N - N') \times 10^{3}$", fontsize=10, labelpad=5)  # Title for y-axis
            axs[1].set_yticks(np.arange(-0.010, 0.0101, 0.010))
            #axs[1].set_yticks(np.arange(-0.0015, 0.0151, 0.005), minor=True)
            axs[1].set_xticks(np.arange(1, 5.5, 1))
            axs[1].tick_params(axis='both', which='both', direction='in', length=4)  # Adjust labelsize as needed
            axs[1].set_xticks(np.arange(1, 5.5, 1))
            axs[1].tick_params(axis='x', which='both', top=True, bottom=True)  # Ticks on top and bottom
            axs[1].tick_params(axis='y', which='both', left=True, right=True)  # Ticks on left and right
            #axs[1].text( 5, 0.825, 'ANN σ: ' + str(round(Sigma_mod, 5)))
            #axs[1].text(5, 0.75, 'LSR  σ: ' + str(round(Sigma_LSQ, 5)))
            axs[1].yaxis.set_major_formatter(FuncFormatter(format_ticks))

   
            # Plot C for experiment, ANN, and LSR
            axs[2].scatter(E,  Experimental_NCS[i][:, 1], facecolor='none', edgecolor = 'blue', s =10,  label='Experiment'  )
            axs[2].plot(E,  Least_Square_NCS[i]['C'], color='black', linestyle='--', label='LSR', linewidth=3 )
            axs[2].plot(E,  ANN_NCS_List[k][i]['C'], color='red', linestyle='--', label='ANN', linewidth=3 )
            axs[2].set_xlim(0.7, 5.8)
            axs[2].set_ylabel('$C$= $Sin2$' + r'$\Psi$' + '$Cos$' + r'$\Delta$', fontsize=10, labelpad=0)  # Title for y-axis

            axs[2].set_ylim(-0.9, -0.3)
            axs[2].set_yticks(np.arange(-0.8, -0.35, 0.2))
            axs[2].tick_params(axis='x', which='both', top=True, bottom=True)  # Ticks on top and bottom
            axs[2].tick_params(axis='y', which='both', left=True, right=True)  # Ticks on left and right
            
            axs[2].tick_params(axis='both', which='major', labelsize=10)  # Adjust labelsize as needed
            axs[2].set_xticks(np.arange(1, 5.5, 1))
            
            

            # Plot C for experiment, ANN, and LSR
            #axs[3].scatter(E,  Experimental_NCS[i][:, 1], facecolor='none', edgecolor = 'blue', s =10,  label='Experiment'  )
            axs[3].plot(E,  Experimental_NCS[i][:, 1] - Least_Square_NCS[i]['C'], color='black', linestyle='--', label='LSR', linewidth=3 )
            axs[3].plot(E,  Experimental_NCS[i][:, 1] - ANN_NCS_List[k][i]['C'], color='red', linestyle='--', label='ANN', linewidth=3 )
            axs[3].set_xlim(0.7, 5.8)
            axs[3].set_ylabel(r"$(C - C') \times 10^{3}$", fontsize=10)  # Title for y-axis
            axs[3].set_ylim(-0.018, 0.018)
            axs[3].set_yticks(np.arange(-0.010, 0.0101, 0.010))
            #axs[3].set_yticks(np.arange(-0.015, 0.0151, 0.005), minor=True)
            axs[3].tick_params(axis='both', which='both', direction='in',  labelsize=10, length=4)  # Adjust labelsize as needed
            axs[3].tick_params(axis='x', which='both', top=True, bottom=True)  # Ticks on top and bottom
            axs[3].tick_params(axis='y', which='both', left=True, right=True)  # Ticks on left and right
            axs[3].yaxis.set_major_formatter(FuncFormatter(format_ticks))
            axs[3].set_xticks(np.arange(1, 5.5, 1))

            # Plot S for experiment, ANN, and LSR
            axs[4].scatter(E,  Experimental_NCS[i][:, 2], facecolor='none', edgecolor = 'blue',  s =10, label='Experiment'  )
            axs[4].plot(E,  Least_Square_NCS[i]['S'], color='black', linestyle='--', label='LSR', linewidth=3 )
            axs[4].plot(E,  ANN_NCS_List[k][i]['S'], color='red', linestyle='--', label='ANN', linewidth=3 )
            axs[4].set_xlim(0.7, 5.8)
            axs[4].set_ylabel('$S$= $Sin2$' + r'$\Psi$' + '$Sin$' + r'$\Delta$' , fontsize=10, labelpad=7 )  # Title for y-axis
            axs[4].set_ylim(-0.15, 1)
            axs[4].set_yticks(np.arange(-0, 0.85, 0.4))
            axs[4].tick_params(axis='both', which='major', labelsize=10)  # Adjust labelsize as needed
            axs[4].tick_params(axis='x', which='both', top=True, bottom=True)  # Ticks on top and bottom
            axs[4].tick_params(axis='y', which='both', left=True, right=True)  # Ticks on left and right
            axs[4].legend(loc='lower right', ncol=1, frameon=False, fontsize=10 )
            axs[4].set_xlabel('Photon Energy (eV)', fontsize=10)  # Title for x-axis
            axs[4].set_xticks(np.arange(1, 5.5, 1))


            # Plot S for experiment, ANN, and LSR
            #axs[5].scatter(E,  Experimental_NCS[i][:, 2], facecolor='none', edgecolor = 'blue',  s =10, label='Experiment'  )
            axs[5].plot(E,  Experimental_NCS[i][:, 2] - Least_Square_NCS[i]['S'], color='black', linestyle='--', label='LSR', linewidth=3 )
            axs[5].plot(E,   Experimental_NCS[i][:, 2] -ANN_NCS_List[k][i]['S'], color='red', linestyle='--', label='ANN', linewidth=3 )
            axs[5].set_xlim(0.7, 5.8)
            axs[5].set_ylabel(r"$(S - S') \times 10^{3}$", fontsize=10)  # Title for y-axis
            axs[5].set_ylim(-0.018, 0.018)
            axs[5].set_yticks(np.arange(-0.010, 0.0101, 0.010))
            #axs[5].set_yticks(np.arange(-0.015, 0.0151, 0.005), minor=True)
            axs[5].tick_params(axis='both', which='both', direction='in',  labelsize=10, length=4)  # Adjust labelsize as needed
            axs[5].tick_params(axis='x', which='both', top=True, bottom=True)  # Ticks on top and bottom
            axs[5].tick_params(axis='y', which='both', left=True, right=True)  # Ticks on left and right
            
            #axs[5].legend(loc='best', ncol=1, frameon=False)
            axs[5].set_xlabel('Photon Energy (eV)', fontsize=10)  # Title for x-axis
            axs[5].yaxis.set_major_formatter(FuncFormatter(format_ticks))
            axs[5].set_xticks(np.arange(1, 5.5, 1))

           
            plt.subplots_adjust(hspace=0, wspace=0.3)

            # Set ticks to point inwards
            for ax in axs:
                ax.tick_params(direction='in', length=4)  # You can adjust 'length' as needed
                for spine in ax.spines.values():
                    spine.set_linewidth(2)  # Set thickness of axis lines
            

            
            if  generate_images: 
                plt.savefig("plot_" + str(i) + ".jpg",  dpi=300, bbox_inches='tight' )  # Saves the plot as a PDF file
           
            # Close the plot to free up memory (optional)
            plt.close()


        

        #Now time to save all of the data 
        Sigma_mod_avg = Sigma_mod_avg / len(ANN_NCS_List[k])
        Sigma_LeastSquare_avg = Sigma_LeastSquare_avg /  len(ANN_NCS_List[k])

        #save average Sigma values 
        Sigma_mod_avg_list.append(Sigma_mod_avg)
        Sigma_lsq_avg_list.append(Sigma_LeastSquare_avg)

        #save each individual value
        Sigma_mod_grand_list.append(Sigma_mod_list)
        Sigma_lsq_grand_list.append( Sigma_LeastSquare_list)
        
        print('Mod ' + str(k + 1) + ' Avg Sigma: ' + str(Sigma_mod_avg))
        print('LSQ ' + 'Avg Sigma: ' + str(Sigma_LeastSquare_avg))


    return (Sigma_mod_grand_list, Sigma_lsq_grand_list)


In [ ]:
############ Generating error between ANN and experiment and LSR and experiment for MV1519 ################

path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO HAVE THESE PLOTS STORED####################

# If you do not want to make plots, set "generate_images=False"
# If you do want to make plots, set "generate_images=True"

MV1519_Sigma, MV1519_LSR_Sigma = NCS_Comparison_ANN3(ANN3_MV1519_NCS, # simulated spectra from ANN
                                                     MV1519[:, :, 1:] , # excperimental data
                                                     LSR_MV1519_NCS,  # traditional LSR data
                                                     6, # number of fit parameters
                                                     path,  # path to save images
                                                     generate_images=False, # makes images when true
                                                     foldername = 'MV1519' # name for the folder created to contain images. 
                                                    )


In [ ]:
############ Generating error between ANN and experiment and LSR and experiment for MV1523 ################

path = r'XXXX' ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO HAVE THESE PLOTS STORED ####################

# If you do not want to make plots, set "generate_images=False"
# If you do want to make plots, set "generate_images=True"

MV1523_Sigma, MV1523_LSR_Sigma = NCS_Comparison_ANN3(ANN3_MV1523_NCS, # simulated spectra from ANN
                                                     MV1523[:, :, 1:] , # excperimental data
                                                     LSR_MV1523_NCS,  # traditional LSR data
                                                     6, # number of fit parameters
                                                     path,  # path to save images
                                                     generate_images=False, # makes images when true
                                                     foldername = 'MV1523' # name for the folder created to contain images. 
                                                    )


In [ ]:
os.chdir(r'XXXX')
df = pd.DataFrame(MV1523_Sigma[-2], columns=['Sigma'])
df.to_csv('ANN3_50000_Sigma_MV1523.csv')


Now it is time to look at the performance of ANN4. 

In [ ]:
# load in mean and standard deviation of training set for each data set

os.chdir(r'XXXX')  ########## PLEASE PUT THE DIRECTORY WHERE YOU STORED THE MEAN AND STANDARD DEVIATIONS FOR EACH DATA SET ####################
############ THE MEAN AND STANDARD DEVIATIONS SHOULD HAVE BEEN MADE IN THE "Model_Development" WORKBOOK ########################

file = 'Data_Set2_10_Mean.csv' # mean for data set of size 10
Data_Set2_10_Mean = pd.read_csv(file) # read in files and pandas dataframe

file = 'Data_Set2_10_std.csv'  # mean standard deviation for data set of size 10
Data_Set2_10_std = pd.read_csv(file) # read in files and pandas dataframe

file = 'Data_Set2_100_Mean.csv' # mean for data set of size 100
Data_Set2_100_Mean = pd.read_csv(file) # read in files and pandas dataframe

file = 'Data_Set2_100_std.csv'  # mean standard deviation for data set of size 100
Data_Set2_100_std = pd.read_csv(file) # read in files and pandas dataframe

file = 'Data_Set2_1000_Mean.csv' # mean for data set of size 1000
Data_Set2_1000_Mean = pd.read_csv(file) # read in files and pandas dataframe

file = 'Data_Set2_1000_std.csv'  # mean standard deviation for data set of size 1000
Data_Set2_1000_std = pd.read_csv(file) # read in files and pandas dataframe

file = 'Data_Set2_10000_Mean.csv' # mean for data set of size 10000
Data_Set2_10000_Mean = pd.read_csv(file) # read in files and pandas dataframe

file = 'Data_Set2_10000_std.csv'  # mean standard deviation for data set of size 10000
Data_Set2_10000_std = pd.read_csv(file) # read in files and pandas dataframe

file = 'Data_Set2_50000_Mean.csv' # mean for data set of size 50000
Data_Set2_50000_Mean = pd.read_csv(file) # read in files and pandas dataframe

file = 'Data_Set2_50000_std.csv'  # mean standard deviation for data set of size 50000
Data_Set2_50000_std = pd.read_csv(file) # read in files and pandas dataframe

file = 'Data_Set2_100000_Mean.csv' # mean for data set of size 50000
Data_Set2_100000_Mean = pd.read_csv(file) # read in files and pandas dataframe

file = 'Data_Set2_100000_std.csv'  # mean standard deviation for data set of size 50000
Data_Set2_100000_std = pd.read_csv(file) # read in files and pandas dataframe


In [ ]:
# Load in all the ANNs 

# move to location of the pre-trained modelsANN3
os.chdir(r"XXXX") ########## PLEASE PUT THE DIRECTORY TO THE FOLDER LABELED "Models" in the XXXX space ####################

# Load in models for the different versions of ANN4
ANN4_10 = load_model('ANN4_10.keras')
ANN4_100 = load_model('ANN4_100.keras')
ANN4_1000 = load_model('ANN4_1000.keras')
ANN4_10000 = load_model('ANN4_10000.keras')
ANN4_50000 = load_model('ANN4_50000v2.keras')
ANN4_100000 = load_model('ANN4_100000.keras')


In [ ]:
# Load in the 10000 test data set.

# Identify the location of the data 
path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY OF THE TEST DATA OF DATA SET 2 in the XXXX space ####################

# Get the data using function "get_data_aSi_CL_E"
test_files, test_data, test_Ep, test_Eg, test_Eo, test_Br, test_Amp, test_Einf, test_BulkT, test_EMA_bool, test_EMAT, test_Sub_bool =  get_data_CL(path)

# Randomly Shuffle the data using function "unison_shuffled_copies"
test_files, test_data, test_Ep, test_Eg, test_Eo, test_Br, test_Amp, test_Einf, test_BulkT, test_EMA_bool, test_EMAT, test_Sub_bool  = unison_shuffled_copies( test_files, test_data, test_Ep, test_Eg, test_Eo, test_Br, test_Amp, test_Einf, test_BulkT, test_EMA_bool, test_EMAT, test_Sub_bool )


# Save test parameters for later analysis
y_test_DS2 = {

    'Ep' :  test_Ep,
    'Eg' :  test_Eg,
    'Eo' :  test_Eo,
    'Br' :  test_Br,
    'Amp' : test_Amp,
    'BulkT': test_BulkT,
    'EMAT' : test_EMAT

}

In [ ]:
# Generate predictions on the test data 

ANN4_10_test = ANN4_10.predict(test_data)
ANN4_100_test = ANN4_100.predict(test_data)
ANN4_1000_test = ANN4_1000.predict(test_data)
ANN4_10000_test = ANN4_10000.predict(test_data)
ANN4_50000_test = ANN4_50000.predict(test_data)
ANN4_100000_test = ANN4_100000.predict(test_data)

In [ ]:
# Unstandardize the data 

ANN4_10_test = UnStandardize_ANN_Output( Data_Set2_10_Mean, Data_Set2_10_std, ANN4_10_test)
ANN4_100_test = UnStandardize_ANN_Output( Data_Set2_100_Mean, Data_Set2_100_std, ANN4_100_test)
ANN4_1000_test = UnStandardize_ANN_Output( Data_Set2_1000_Mean, Data_Set2_1000_std, ANN4_1000_test)
ANN4_10000_test = UnStandardize_ANN_Output( Data_Set2_10000_Mean, Data_Set2_10000_std, ANN4_10000_test)
ANN4_50000_test = UnStandardize_ANN_Output( Data_Set2_50000_Mean, Data_Set2_50000_std, ANN4_50000_test)
ANN4_100000_test = UnStandardize_ANN_Output( Data_Set2_100000_Mean, Data_Set2_100000_std, ANN4_100000_test)



In [ ]:
# Now generate predictions on the experimental data 

ANN4_10_MV1530 = ANN4_10.predict(MV1530)
ANN4_100_MV1530 = ANN4_100.predict(MV1530)
ANN4_1000_MV1530 = ANN4_1000.predict(MV1530)
ANN4_10000_MV1530 = ANN4_10000.predict(MV1530)
ANN4_50000_MV1530 = ANN4_50000.predict(MV1530)
ANN4_100000_MV1530 = ANN4_100000.predict(MV1530)

In [ ]:
# Now unstandardize the predictions

ANN4_10_MV1530 = UnStandardize_ANN_Output( Data_Set2_10_Mean, Data_Set2_10_std, ANN4_10_MV1530)
ANN4_100_MV1530 = UnStandardize_ANN_Output( Data_Set2_100_Mean, Data_Set2_100_std, ANN4_100_MV1530)
ANN4_1000_MV1530 = UnStandardize_ANN_Output( Data_Set2_1000_Mean, Data_Set2_1000_std, ANN4_1000_MV1530)
ANN4_10000_MV1530 = UnStandardize_ANN_Output( Data_Set2_10000_Mean, Data_Set2_10000_std, ANN4_10000_MV1530)
ANN4_50000_MV1530 = UnStandardize_ANN_Output( Data_Set2_50000_Mean, Data_Set2_50000_std, ANN4_50000_MV1530)
ANN4_100000_MV1530 = UnStandardize_ANN_Output( Data_Set2_100000_Mean, Data_Set2_100000_std, ANN4_100000_MV1530)


# Now store the predictions
os.chdir(r'XXXX')  ########## PLEASE PUT THE DIRECTORY FOR WHERE YOU WOULD LIKE THE ANN PREDICTIONS TO BE STORED ####################

df = pd.DataFrame(ANN4_10_MV1530)
df.to_csv('ANN4_10_MV1530.csv')

df = pd.DataFrame(ANN4_100_MV1530)
df.to_csv('ANN4_100_MV1530.csv')

df = pd.DataFrame(ANN4_1000_MV1530)
df.to_csv('ANN4_1000_MV1530.csv')

df = pd.DataFrame(ANN4_10000_MV1530)
df.to_csv('ANN4_10000_MV1530.csv')

df = pd.DataFrame(ANN4_50000_MV1530)
df.to_csv('ANN4_50000_MV1530.csv')

df = pd.DataFrame(ANN4_100000_MV1530)
df.to_csv('ANN4_100000_MV1530.csv')

Now the predictions ANN4 are ready. Ellipsometric spectra will be generated for each prediction and compared to the experimental data by calculating the unweighted error function between ANN and experiment. The traditional least-square regression (LSR) unweighted error function will also be calculated. 

In [ ]:
# Generate ellipsometric spectra from LSR analysis

LSR_MV1530_NCS =[]
MV1530_CL_List =[]
for i in range(len(MV1530_params['Amp'])):
    
    #first get Tacu-Lorentz Oscillator parameters
    E_inf = 1 # range fixed to 1
    Ep  = float(MV1530_params['Ep'][i])
    Amp  = float(MV1530_params['Amp'][i])
    Br =   float(MV1530_params['Br'][i])
    Eo =   float(MV1530_params['Eo'][i])
    Eg =   float(MV1530_params['Eg'][i])
    Et = 0 #this term being 0 effectivly neglects the Urbach energy and simplifies the equations.
    Egt = Eg + Et
    
    # Now we generate our Cody-Lorentz material
    CL = Get_CL_Material(E, Ep, Eg, Eo, Br, Amp, Egt, E_inf, wv)
    EMA = Bruggeman_EMA( CL, Void, 0.5)

    EMA_Substrate =  Bruggeman_EMA( CL, SLG , 0.5 )
    
    MV1530_CL_List.append(CL)
    
    #Generate Thicknesses
    Bulk_Thickness = float(MV1530_params['BulkT'][i])
    EMA_Thickness =  float(MV1530_params['EMAT'][i])

    
    #define Structure 
    Structure = [Void, EMA, CL, EMA_Substrate, SLG]
    #Define material thicknesses
    Mat_Thick = np.array([EMA_Thickness, Bulk_Thickness, 3 ] )
    #Define angle of incidence
    Theta_Incident = 64.93
    df = SE_Sim(Structure, Theta_Incident, Mat_Thick,  write_data=False, NCS=True)
    df = df.drop(df.columns[0], axis=1)
    LSR_MV1530_NCS.append(df)


Now define a function to generate ellipsometric spectra for the ANN4 outputs

In [ ]:
# define function to generate ellipsometric spectra for SLG substrate structure

def NCS_Genorator_ANN4(ANN_List, E, wv): 

    """
    The goal of this function is to generate ellipsometric spectra for each ANN output in the ANN_List. 
    ANN_List should be a list of dictionaries. Each dictionary contains a set of predictions for a given ANN model
    """
    NCS_List = [] #stores the ellipsometric specta
    CL_List =[]
    for j in range(len(ANN_List)): # iterate over the entire list of ANN output
        print(j) # check if working
        modX_NCS_List = [] # list containing values for a specific model
        modX_CL_List =[]   
        for i in range(len(ANN_List[j]['Eg'])): #iterate over each prediction for a given ANN
    
            #first get Cody-Loretnz Oscillator parameters
            E_inf = 1 # range fixed to 1
            Ep  = ANN_List[j]['Ep'][i]
            Amp  = ANN_List[j]['Amp'][i]
            Br =   ANN_List[j]['Br'][i]
            Eo =   ANN_List[j]['Eo'][i]
            Eg =   ANN_List[j]['Eg'][i]
            Et = 0 #this term being 0 effectivly neglects the Urbach energy and simplifies the equations.
            Egt = Eg + Et
        
            # Now we generate our Cody-Lorentz material
            CL = Get_CL_Material(E, Ep, Eg, Eo, Br, Amp, Egt, E_inf, wv)
            modX_CL_List.append(CL)
            #Now we define our structure
                
            #Void_Frac = 0.5
        
            #Generate EMA
            EMA = Bruggeman_EMA( CL, Void, 0.5)
            EMA_Substrate =  Bruggeman_EMA( CL, SLG , 0.5 )
        
            #Generate Thicknesses
            Bulk_Thickness = ANN_List[j]['BulkT'][i]
            EMA_Thickness =  ANN_List[j]['EMAT'][i]
              
        
            #define Structure 
            Structure = [Void, EMA, CL, EMA_Substrate, SLG]
            Mat_Thick = np.array([EMA_Thickness, Bulk_Thickness, 3 ] )
            Theta_Incident = 64.93
            df = SE_Sim(Structure, Theta_Incident, Mat_Thick,  write_data=False, NCS=True)
            df = df.drop(df.columns[0], axis=1)
            modX_NCS_List.append(df) # store prediction in the list for that model
        
        NCS_List.append(modX_NCS_List) # store that entire model's predictions as 1 term in this list
        CL_List.append(modX_CL_List)

    return(NCS_List, CL_List)
        
                  

In [ ]:
# Put outputs in lists 
MV1530_list = [ANN4_10_MV1530, ANN4_100_MV1530, ANN4_1000_MV1530, ANN4_10000_MV1530, ANN4_50000_MV1530, ANN4_100000_MV1530]

# Calculate ellipsometric spectra
ANN4_MV1530_NCS, ANN4_MV1530_CL = NCS_Genorator_ANN4(MV1530_list, E, wv)

Now we can compare these spectra to the experimental data and the least-square regression. The unweighted error function will be generate for each point to help make comparisons. The average error will be desplayed. A function that makes these comparisons is defined to compare the data from ANN4. This function will also export figures containing this data.

Two different functions will be defined for ANN4. Each one generating different graphs, but otherwise function the same.

In [ ]:
def NCS_Comparison_ANN4_v1(ANN_NCS_List, Experimental_NCS, Least_Square_NCS, param_num, path, generate_images=False, foldername = ''):

    """
    Goal of function is to iterate over every ANN NCS list and compare each ANN list to the corresponsinf experimental 
    and least-square regression analysis. 

    If generate_images is set to True, then pictures for wach spectra will be generated.
    """

    #iterate over the ANN_NCS_list

    Sigma_mod_avg_list = [] # list to store the average value of sigma for each model
    Sigma_mod_grand_list =[] # list to store every value for sigma for each model

    Sigma_lsq_avg_list =[]
    Sigma_lsq_grand_list = []

    for k in range(len(ANN_NCS_List)): #iterate for each ANN model

        #create path to store images if  generate_images=True
        folder = str(  'ANN4_' + str(10**(k + 1)) + '_' + foldername)
        if k == 4: 
            folder = 'ANN4_' + str(50000)  + '_' + foldername
        if k == 5: 
            folder = 'ANN4_' + str(100000)  + '_' + foldername
        
        if generate_images:
            os.makedirs(path + folder, exist_ok=True)
            os.chdir(path + folder)

        #initalize values
        Sigma_mod_list =[]
        Sigma_mod_avg = 0 #stores the average value of Sigma 
        
        Sigma_LeastSquare_avg = 0 # stores the average value of Sigma for least square regression 
        Sigma_LeastSquare_list = []

        for i in range(len(ANN_NCS_List[k])):# iterate for each measurement. In this case, there are 81 measurements 

            #Calcualte Sigma between ANN prediction N,C,S spectra and real SE data
            Sigma_mod = SE_Sigma(Experimental_NCS[i], ANN_NCS_List[k][i], param_num)
            Sigma_mod_avg = Sigma_mod_avg + Sigma_mod
            Sigma_mod_list.append(Sigma_mod)
            
            #plt.text(5, 1, 'ANN Model σ: ' + str(round(Sigma_mod, 5)) , fontsize=10, ha='center')
            
            
            #Calcualte MSE between LSR N,C,S spectra and real SE data
            Sigma_LSQ = SE_Sigma(Experimental_NCS[i], Least_Square_NCS[i], param_num)
            Sigma_LeastSquare_avg = Sigma_LeastSquare_avg + Sigma_LSQ
            Sigma_LeastSquare_list.append(Sigma_LSQ)

            def format_ticks(x, _):
                return f'{x * 1000:.0f}'  # Multiply by scale_factor

            
            # Create a figure with 3 stacked subplots
            plt.rcParams['axes.unicode_minus'] = True

            fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(10, 5)) 

            axs = axs.flatten()

            # Plot N for experiment, ANN, and LSR
            axs[0].scatter(E,  Experimental_NCS[i][:, 0], facecolor='none', edgecolor = 'blue', s =10,   label='Experiment'  )
            axs[0].plot(E,  Least_Square_NCS[i]['N'], color='black', linestyle='--', label='LSR', linewidth=3 )
            axs[0].plot(E,  ANN_NCS_List[k][i]['N'], color='red', linestyle='--', label='ANN', linewidth=3 )
            axs[0].set_xlim(0.7, 5.8)
            axs[0].set_xticks(np.arange(1, 5.5, 1))

            axs[0].set_ylim(0.2, 1)
            axs[0].set_yticks(np.arange(0.3, 0.95, 0.3))
            
            axs[0].set_ylabel('$N$= $Cos2$' + r'$\Psi$ ', fontsize=10, labelpad=9)  # Title for y-axis
            axs[0].text(4.15, 0.85, 'ANN σ: ' + str("{:.4f}".format(Sigma_mod)), fontsize =10)
            axs[0].text(4.15, 0.75, 'LSR  σ: ' + str("{:.4f}".format(Sigma_LSQ)), fontsize =10 )
            axs[0].tick_params(axis='both', which='major', labelsize=10)  # Adjust labelsize as needed
            axs[0].tick_params(axis='x', which='both', top=True, bottom=True)  # Ticks on top and bottom
            axs[0].tick_params(axis='y', which='both', left=True, right=True)  # Ticks on left and right

   
            # Plot N for experiment, ANN, and LSR
            #axs[1].scatter(E,  Experimental_NCS[i][:, 0], facecolor='none', edgecolor = 'blue', s =10,   label='Experiment'  )
            axs[1].plot(E,  Experimental_NCS[i][:, 0] - Least_Square_NCS[i]['N'], color='black', linestyle='--', label='LSR', linewidth=3 )
            axs[1].plot(E, Experimental_NCS[i][:, 0] - ANN_NCS_List[k][i]['N'], color='red', linestyle='--', label='ANN', linewidth=3 )
            axs[1].set_xlim(0.7, 5.8)
            axs[1].set_ylim(-0.025, 0.025)
            axs[1].set_ylabel(r"$(N - N') \times 10^{3}$", fontsize=10, labelpad=8)  # Title for y-axis
            axs[1].set_yticks(np.arange(-0.015, 0.0151, 0.015))
            #axs[1].set_yticks(np.arange(-0.0015, 0.0151, 0.005), minor=True)
            axs[1].set_xticks(np.arange(1, 5.5, 1))
            axs[1].tick_params(axis='both', which='major', labelsize=10)  # Adjust labelsize as needed
            axs[1].set_xticks(np.arange(1, 5.5, 1))
            axs[1].tick_params(axis='x', which='both', top=True, bottom=True)  # Ticks on top and bottom
            axs[1].tick_params(axis='y', which='both', left=True, right=True)  # Ticks on left and right
            #axs[1].text( 5, 0.825, 'ANN σ: ' + str(round(Sigma_mod, 5)))
            #axs[1].text(5, 0.75, 'LSR  σ: ' + str(round(Sigma_LSQ, 5)))
            axs[1].yaxis.set_major_formatter(FuncFormatter(format_ticks))

   
            # Plot C for experiment, ANN, and LSR
            axs[2].scatter(E,  Experimental_NCS[i][:, 1], facecolor='none', edgecolor = 'blue', s =10,  label='Experiment'  )
            axs[2].plot(E,  Least_Square_NCS[i]['C'], color='black', linestyle='--', label='LSR', linewidth=3 )
            axs[2].plot(E,  ANN_NCS_List[k][i]['C'], color='red', linestyle='--', label='ANN', linewidth=3 )
            axs[2].set_xlim(0.7, 5.8)
            axs[2].set_ylabel('$C$= $Sin2$' + r'$\Psi$' + '$Cos$' + r'$\Delta$', fontsize=10, labelpad=0)  # Title for y-axis

            axs[2].set_ylim(-1, 0.7)
            axs[2].set_yticks(np.arange(-0.8, 0.7, 0.6))
            axs[2].tick_params(axis='x', which='both', top=True, bottom=True)  # Ticks on top and bottom
            axs[2].tick_params(axis='y', which='both', left=True, right=True)  # Ticks on left and right
            
            axs[2].tick_params(axis='both', which='major', labelsize=10)  # Adjust labelsize as needed
            axs[2].set_xticks(np.arange(1, 5.5, 1))
            
            

            # Plot C for experiment, ANN, and LSR
            #axs[3].scatter(E,  Experimental_NCS[i][:, 1], facecolor='none', edgecolor = 'blue', s =10,  label='Experiment'  )
            axs[3].plot(E,  Experimental_NCS[i][:, 1] - Least_Square_NCS[i]['C'], color='black', linestyle='--', label='LSR', linewidth=3 )
            axs[3].plot(E,  Experimental_NCS[i][:, 1] - ANN_NCS_List[k][i]['C'], color='red', linestyle='--', label='ANN', linewidth=3 )
            axs[3].set_xlim(0.7, 5.8)
            axs[3].set_ylabel(r"$(C - C') \times 10^{3}$", fontsize=10, labelpad=0)  # Title for y-axis
            axs[3].set_ylim(-0.15, 0.15)
            axs[3].set_yticks(np.arange(-0.1, 0.11, 0.1))
            #axs[3].set_yticks(np.arange(-0.1, 0.11, 0.05), minor = True)
            axs[3].tick_params(axis='both', which='major', labelsize=10)  # Adjust labelsize as needed
            axs[3].tick_params(axis='both', which='major', labelsize=10)  # Adjust labelsize as needed
            axs[3].tick_params(axis='x', which='both', top=True, bottom=True)  # Ticks on top and bottom
            axs[3].tick_params(axis='y', which='both', left=True, right=True)  # Ticks on left and right
            axs[3].yaxis.set_major_formatter(FuncFormatter(format_ticks))
            axs[3].set_xticks(np.arange(1, 5.5, 1))

            # Plot S for experiment, ANN, and LSR
            axs[4].scatter(E,  Experimental_NCS[i][:, 2], facecolor='none', edgecolor = 'blue',  s =10, label='Experiment'  )
            axs[4].plot(E,  Least_Square_NCS[i]['S'], color='black', linestyle='--', label='LSR', linewidth=3 )
            axs[4].plot(E,  ANN_NCS_List[k][i]['S'], color='red', linestyle='--', label='ANN', linewidth=3 )
            axs[4].set_xlim(0.7, 5.8)
            axs[4].set_ylabel('$S$= $Sin2$' + r'$\Psi$' + '$Sin$' + r'$\Delta$' , fontsize=10, labelpad=0 )  # Title for y-axis
            axs[4].set_ylim(-1, 1)
            axs[4].set_yticks(np.arange(-0.8, 0.85, 0.8))
            axs[4].tick_params(axis='both', which='major', labelsize=10)  # Adjust labelsize as needed
            axs[4].tick_params(axis='x', which='both', top=True, bottom=True)  # Ticks on top and bottom
            axs[4].tick_params(axis='y', which='both', left=True, right=True)  # Ticks on left and right
            axs[4].legend(loc='lower right', ncol=1, frameon=False, fontsize=10 )
            axs[4].set_xlabel('Photon Energy (eV)', fontsize=10)  # Title for x-axis
            axs[4].set_xticks(np.arange(1, 5.5, 1))


            # Plot S for experiment, ANN, and LSR
            #axs[5].scatter(E,  Experimental_NCS[i][:, 2], facecolor='none', edgecolor = 'blue',  s =10, label='Experiment'  )
            axs[5].plot(E,  Experimental_NCS[i][:, 2] - Least_Square_NCS[i]['S'], color='black', linestyle='--', label='LSR', linewidth=3 )
            axs[5].plot(E,   Experimental_NCS[i][:, 2] -ANN_NCS_List[k][i]['S'], color='red', linestyle='--', label='ANN', linewidth=3 )
            axs[5].set_xlim(0.7, 5.8)
            axs[5].set_ylabel(r"$(S - S') \times 10^{3}$", fontsize=10, labelpad=0 )  # Title for y-axis
            axs[5].set_ylim(-0.15, 0.15)
            axs[5].set_yticks(np.arange(-0.1, 0.11, 0.1))
            #axs[5].set_yticks(np.arange(-0.1, 0.11, 0.05), minor = True)
            axs[5].tick_params(axis='both', which='major', labelsize=10)  # Adjust labelsize as needed
            axs[5].tick_params(axis='x', which='both', top=True, bottom=True)  # Ticks on top and bottom
            axs[5].tick_params(axis='y', which='both', left=True, right=True)  # Ticks on left and right
            
            #axs[5].legend(loc='best', ncol=1, frameon=False)
            axs[5].set_xlabel('Photon Energy (eV)', fontsize=10)  # Title for x-axis
            axs[5].yaxis.set_major_formatter(FuncFormatter(format_ticks))
            axs[5].set_xticks(np.arange(1, 5.5, 1))

           
            plt.subplots_adjust(hspace=0, wspace=0.3)
            

            # Set ticks to point inwards
            for ax in axs:
                ax.tick_params(direction='in', length=4)  # You can adjust 'length' as needed
                for spine in ax.spines.values():
                    spine.set_linewidth(2)  # Set thickness of axis lines
            

            
            if  generate_images: 
                plt.savefig("plot_" + str(i) + ".jpg",  dpi=300, bbox_inches='tight' )  # Saves the plot as a PDF file
           
            # Close the plot to free up memory (optional)
            plt.close()


        

        #Now time to save all of the data 
        Sigma_mod_avg = Sigma_mod_avg / len(ANN_NCS_List[k])
        Sigma_LeastSquare_avg = Sigma_LeastSquare_avg /  len(ANN_NCS_List[k])

        #save average Sigma values 
        Sigma_mod_avg_list.append(Sigma_mod_avg)
        Sigma_lsq_avg_list.append(Sigma_LeastSquare_avg)

        #save each individual value
        Sigma_mod_grand_list.append(Sigma_mod_list)
        Sigma_lsq_grand_list.append( Sigma_LeastSquare_list)
        
        print('Mod ' + str(k + 1) + ' Avg Sigma: ' + str(Sigma_mod_avg))
        print('LSQ ' + 'Avg Sigma: ' + str(Sigma_LeastSquare_avg))


    return (Sigma_mod_grand_list, Sigma_lsq_grand_list)


In [ ]:
def NCS_Comparison_ANN4_v2(ANN_NCS_List, Experimental_NCS, Least_Square_NCS, param_num, path, generate_images=False, foldername = ''):

    """
    Goal of function is to iterate over every ANN NCS list and compare each ANN list to the corresponsinf experimental 
    and least-square regression analysis. 

    If generate_images is set to True, then pictures for wach spectra will be generated.
    """

    #iterate over the ANN_NCS_list

    Sigma_mod_avg_list = [] # list to store the average value of sigma for each model
    Sigma_mod_grand_list =[] # list to store every value for sigma for each model

    Sigma_lsq_avg_list =[]
    Sigma_lsq_grand_list = []

    for k in range(len(ANN_NCS_List)): #iterate for each ANN model

        #create path to store images if  generate_images=True
        folder = str(  'ANN4_' + str(10**(k + 1)) + '_' + foldername)
        if k == 4: 
            folder = 'ANN4_' + str(50000)  + '_' + foldername
        if k == 5: 
            folder = 'ANN4_' + str(100000)  + '_' + foldername
        
        if generate_images:
            os.makedirs(path + folder, exist_ok=True)
            os.chdir(path + folder)

        #initalize values
        Sigma_mod_list =[]
        Sigma_mod_avg = 0 #stores the average value of Sigma 
        
        Sigma_LeastSquare_avg = 0 # stores the average value of Sigma for least square regression 
        Sigma_LeastSquare_list = []

        for i in range(len(ANN_NCS_List[k])):# iterate for each measurement. In this case, there are 81 measurements 

            #Calcualte Sigma between ANN prediction N,C,S spectra and real SE data
            Sigma_mod = SE_Sigma(Experimental_NCS[i], ANN_NCS_List[k][i], param_num)
            Sigma_mod_avg = Sigma_mod_avg + Sigma_mod
            Sigma_mod_list.append(Sigma_mod)
            
            #plt.text(5, 1, 'ANN Model σ: ' + str(round(Sigma_mod, 5)) , fontsize=10, ha='center')
            
            
            #Calcualte MSE between LSR N,C,S spectra and real SE data
            Sigma_LSQ = SE_Sigma(Experimental_NCS[i], Least_Square_NCS[i], param_num)
            Sigma_LeastSquare_avg = Sigma_LeastSquare_avg + Sigma_LSQ
            Sigma_LeastSquare_list.append(Sigma_LSQ)

            def format_ticks(x, _):
                return f'{x * 1000:.0f}'  # Multiply by scale_factor

            
            # Create a figure with 3 stacked subplots
            fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(10, 5)) 

            axs = axs.flatten()

            # Plot N for experiment, ANN, and LSR
            axs[0].scatter(E,  Experimental_NCS[i][:, 0], facecolor='none', edgecolor = 'blue', s =10,   label='Experiment'  )
            axs[0].plot(E,  Least_Square_NCS[i]['N'], color='black', linestyle='--', label='LSR', linewidth=3 )
            axs[0].plot(E,  ANN_NCS_List[k][i]['N'], color='red', linestyle='--', label='ANN', linewidth=3 )
            axs[0].set_xlim(0.7, 5.8)
            axs[0].set_xticks(np.arange(1, 5.5, 1))

            axs[0].set_ylim(0.2, 1)
            axs[0].set_yticks(np.arange(0.3, 0.95, 0.3))
            
            axs[0].set_ylabel('$N$= $Cos2$' + r'$/Psi$ ', fontsize=10, labelpad=8)  # Title for y-axis
            axs[0].text(4.15, 0.85, 'ANN σ: ' + str("{:.4f}".format(Sigma_mod)), fontsize =10)
            axs[0].text(4.15, 0.75, 'LSR  σ: ' + str("{:.4f}".format(Sigma_LSQ)), fontsize =10 )
            axs[0].tick_params(axis='both', which='major', labelsize=10)  # Adjust labelsize as needed
            axs[0].tick_params(axis='x', which='both', top=True, bottom=True)  # Ticks on top and bottom
            axs[0].tick_params(axis='y', which='both', left=True, right=True)  # Ticks on left and right

   
            # Plot N for experiment, ANN, and LSR
            #axs[1].scatter(E,  Experimental_NCS[i][:, 0], facecolor='none', edgecolor = 'blue', s =10,   label='Experiment'  )
            axs[1].plot(E,  Experimental_NCS[i][:, 0] - Least_Square_NCS[i]['N'], color='black', linestyle='--', label='LSR', linewidth=3 )
            axs[1].plot(E, Experimental_NCS[i][:, 0] - ANN_NCS_List[k][i]['N'], color='red', linestyle='--', label='ANN', linewidth=3 )
            axs[1].set_xlim(0.7, 5.8)
            axs[1].set_ylim(-0.018, 0.018)
            axs[1].set_ylabel(r"$(N - N') /times 10^{3}$", fontsize=10, labelpad=8)  # Title for y-axis
            axs[1].set_yticks(np.arange(-0.010, 0.0101, 0.010))
            #axs[1].set_yticks(np.arange(-0.0015, 0.0151, 0.005), minor=True)
            axs[1].set_xticks(np.arange(1, 5.5, 1))
            axs[1].tick_params(axis='both', which='major', labelsize=10)  # Adjust labelsize as needed
            axs[1].set_xticks(np.arange(1, 5.5, 1))
            axs[1].tick_params(axis='x', which='both', top=True, bottom=True)  # Ticks on top and bottom
            axs[1].tick_params(axis='y', which='both', left=True, right=True)  # Ticks on left and right
            #axs[1].text( 5, 0.825, 'ANN σ: ' + str(round(Sigma_mod, 5)))
            #axs[1].text(5, 0.75, 'LSR  σ: ' + str(round(Sigma_LSQ, 5)))
            axs[1].yaxis.set_major_formatter(FuncFormatter(format_ticks))

   
            # Plot C for experiment, ANN, and LSR
            axs[2].scatter(E,  Experimental_NCS[i][:, 1], facecolor='none', edgecolor = 'blue', s =10,  label='Experiment'  )
            axs[2].plot(E,  Least_Square_NCS[i]['C'], color='black', linestyle='--', label='LSR', linewidth=3 )
            axs[2].plot(E,  ANN_NCS_List[k][i]['C'], color='red', linestyle='--', label='ANN', linewidth=3 )
            axs[2].set_xlim(0.7, 5.8)
            axs[2].set_ylabel('$C$= $Sin2$' + r'$/Psi$' + '$Cos$' + r'$/Delta$', fontsize=10, labelpad=0)  # Title for y-axis

            axs[2].set_ylim(-1, 0.7)
            axs[2].set_yticks(np.arange(-0.8, 0.7, 0.6))
            axs[2].tick_params(axis='x', which='both', top=True, bottom=True)  # Ticks on top and bottom
            axs[2].tick_params(axis='y', which='both', left=True, right=True)  # Ticks on left and right
            
            axs[2].tick_params(axis='both', which='major', labelsize=10)  # Adjust labelsize as needed
            axs[2].set_xticks(np.arange(1, 5.5, 1))
            
            

            # Plot C for experiment, ANN, and LSR
            #axs[3].scatter(E,  Experimental_NCS[i][:, 1], facecolor='none', edgecolor = 'blue', s =10,  label='Experiment'  )
            axs[3].plot(E,  Experimental_NCS[i][:, 1] - Least_Square_NCS[i]['C'], color='black', linestyle='--', label='LSR', linewidth=3 )
            axs[3].plot(E,  Experimental_NCS[i][:, 1] - ANN_NCS_List[k][i]['C'], color='red', linestyle='--', label='ANN', linewidth=3 )
            axs[3].set_xlim(0.7, 5.8)
            axs[3].set_ylabel(r"$(C - C') /times 10^{3}$", fontsize=10)  # Title for y-axis
            axs[3].set_ylim(-0.018, 0.018)
            axs[3].set_yticks(np.arange(-0.010, 0.0101, 0.010))
            #axs[3].set_yticks(np.arange(-0.1, 0.11, 0.05), minor = True)
            axs[3].tick_params(axis='both', which='major', labelsize=10)  # Adjust labelsize as needed
            axs[3].tick_params(axis='both', which='major', labelsize=10)  # Adjust labelsize as needed
            axs[3].tick_params(axis='x', which='both', top=True, bottom=True)  # Ticks on top and bottom
            axs[3].tick_params(axis='y', which='both', left=True, right=True)  # Ticks on left and right
            axs[3].yaxis.set_major_formatter(FuncFormatter(format_ticks))
            axs[3].set_xticks(np.arange(1, 5.5, 1))

            # Plot S for experiment, ANN, and LSR
            axs[4].scatter(E,  Experimental_NCS[i][:, 2], facecolor='none', edgecolor = 'blue',  s =10, label='Experiment'  )
            axs[4].plot(E,  Least_Square_NCS[i]['S'], color='black', linestyle='--', label='LSR', linewidth=3 )
            axs[4].plot(E,  ANN_NCS_List[k][i]['S'], color='red', linestyle='--', label='ANN', linewidth=3 )
            axs[4].set_xlim(0.7, 5.8)
            axs[4].set_ylabel('$S$= $Sin2$' + r'$/Psi$' + '$Sin$' + r'$/Delta$' , fontsize=10, labelpad=0 )  # Title for y-axis
            axs[4].set_ylim(-1, 1)
            axs[4].set_yticks(np.arange(-0.8, 0.85, 0.8))
            axs[4].tick_params(axis='both', which='major', labelsize=10)  # Adjust labelsize as needed
            axs[4].tick_params(axis='x', which='both', top=True, bottom=True)  # Ticks on top and bottom
            axs[4].tick_params(axis='y', which='both', left=True, right=True)  # Ticks on left and right
            axs[4].legend(loc='lower right', ncol=1, frameon=False, fontsize=10 )
            axs[4].set_xlabel('Photon Energy (eV)', fontsize=10)  # Title for x-axis
            axs[4].set_xticks(np.arange(1, 5.5, 1))


            # Plot S for experiment, ANN, and LSR
            #axs[5].scatter(E,  Experimental_NCS[i][:, 2], facecolor='none', edgecolor = 'blue',  s =10, label='Experiment'  )
            axs[5].plot(E,  Experimental_NCS[i][:, 2] - Least_Square_NCS[i]['S'], color='black', linestyle='--', label='LSR', linewidth=3 )
            axs[5].plot(E,   Experimental_NCS[i][:, 2] -ANN_NCS_List[k][i]['S'], color='red', linestyle='--', label='ANN', linewidth=3 )
            axs[5].set_xlim(0.7, 5.8)
            axs[5].set_ylabel(r"$(S - S') /times 10^{3}$", fontsize=10)  # Title for y-axis
            axs[5].set_ylim(-0.018, 0.018)
            axs[5].set_yticks(np.arange(-0.010, 0.0101, 0.010))
            #axs[5].set_yticks(np.arange(-0.1, 0.11, 0.05), minor = True)
            axs[5].tick_params(axis='both', which='major', labelsize=10)  # Adjust labelsize as needed
            axs[5].tick_params(axis='x', which='both', top=True, bottom=True)  # Ticks on top and bottom
            axs[5].tick_params(axis='y', which='both', left=True, right=True)  # Ticks on left and right
            
            #axs[5].legend(loc='best', ncol=1, frameon=False)
            axs[5].set_xlabel('Photon Energy (eV)', fontsize=10)  # Title for x-axis
            axs[5].yaxis.set_major_formatter(FuncFormatter(format_ticks))
            axs[5].set_xticks(np.arange(1, 5.5, 1))

           
            plt.subplots_adjust(hspace=0, wspace=0.3)

            # Set ticks to point inwards
            for ax in axs:
                ax.tick_params(direction='in', length=4)  # You can adjust 'length' as needed
                for spine in ax.spines.values():
                    spine.set_linewidth(2)  # Set thickness of axis lines
            

            
            if  generate_images: 
                plt.savefig("plot_" + str(i) + ".jpg",  dpi=300, bbox_inches='tight' )  # Saves the plot as a PDF file
           
            # Close the plot to free up memory (optional)
            plt.close()


        

        #Now time to save all of the data 
        Sigma_mod_avg = Sigma_mod_avg / len(ANN_NCS_List[k])
        Sigma_LeastSquare_avg = Sigma_LeastSquare_avg /  len(ANN_NCS_List[k])

        #save average Sigma values 
        Sigma_mod_avg_list.append(Sigma_mod_avg)
        Sigma_lsq_avg_list.append(Sigma_LeastSquare_avg)

        #save each individual value
        Sigma_mod_grand_list.append(Sigma_mod_list)
        Sigma_lsq_grand_list.append( Sigma_LeastSquare_list)
        
        print('Mod ' + str(k + 1) + ' Avg Sigma: ' + str(Sigma_mod_avg))
        print('LSQ ' + 'Avg Sigma: ' + str(Sigma_LeastSquare_avg))


    return (Sigma_mod_grand_list, Sigma_lsq_grand_list)


In [ ]:
############ Generating error between ANN and experiment and LSR and experiment for MV1530 ################

path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO HAVE THESE PLOTS STORED####################

# If you do not want to make plots, set "generate_images=False"
# If you do want to make plots, set "generate_images=True"

MV1530_Sigma, MV1530_LSR_Sigma = NCS_Comparison_ANN4_v1(ANN4_MV1530_NCS, # simulated spectra from ANN
                                                        MV1530[:, :, 1:], # excperimental data
                                                        LSR_MV1530_NCS, # traditional LSR data
                                                        7, # number of fit parameters
                                                        path, # path to save images
                                                        generate_images=False, # makes images when true
                                                        foldername = 'MV1530V1'# name for the folder created to contain images. 
                                                       )


In [ ]:
os.chdir(r'XXXX')
df = pd.DataFrame(MV1530_Sigma[-2], columns=['Sigma'])
df.to_csv('ANN4_50000_Sigma_MV1530.csv')


In [ ]:
############ Generating error between ANN and experiment and LSR and experiment for MV1530 ################

path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO HAVE THESE PLOTS STORED####################

# If you do not want to make plots, set "generate_images=False"
# If you do want to make plots, set "generate_images=True"

MV1530_Sigma, MV1530_LSR_Sigma = NCS_Comparison_ANN4_v2(ANN4_MV1530_NCS, # simulated spectra from ANN
                                                        MV1530[:, :, 1:], # excperimental data
                                                        LSR_MV1530_NCS, # traditional LSR data
                                                        7, # number of fit parameters
                                                        path, # path to save images
                                                        generate_images=False, # makes images when true
                                                        foldername = 'MV1530V2'# name for the folder created to contain images. 
                                                       )


In [ ]:
#### Finding extreme values for the MV1519 data when predicted by ANN3_50000 #########

data = MV1519_Sigma[-2]
median_value = statistics.median(data)
median_index = data.index(median_value)
print(f'MV1519 Median Sigma: {median_value}, Index Sigma: {median_index}')

# Finding minimum and maximum values
min_value = min(data)
max_value = max(data)

# Finding indices of minimum and maximum values
min_index = data.index(min_value)
max_index = data.index(max_value)

print(f'MV1519 Min Sigma: {min_value}, Index Sigma: {min_index}')
print(f'MV1519 Max Sigma: {max_value}, Index Sigma: {max_index}')


#### Finding extreme values for the MV1523 data when predicted by ANN3_50000 #########

data = MV1523_Sigma[-2]
median_value = statistics.median(data)
median_index = data.index(median_value)
print(f'MV1523 Median Sigma: {median_value}, Index Sigma: {median_index}')

# Finding minimum and maximum values
min_value = min(data)
max_value = max(data)

# Finding indices of minimum and maximum values
min_index = data.index(min_value)
max_index = data.index(max_value)

print(f'MV1523 Min Sigma: {min_value}, Index Sigma: {min_index}')
print(f'MV1523 Max Sigma: {max_value}, Index Sigma: {max_index}')

#### Finding extreme values for the MV1530 data when predicted by ANN3_50000 #########

data = MV1530_Sigma[-2]
median_value = statistics.median(data)
median_index = data.index(median_value)
print(f'MV1530 Median Sigma: {median_value}, Index Sigma: {median_index}')

# Finding minimum and maximum values
min_value = min(data)
max_value = max(data)

# Finding indices of minimum and maximum values
min_index = data.index(min_value)
max_index = data.index(max_value)

print(f'MV1530 Min Sigma: {min_value}, Index Sigma: {min_index}')
print(f'MV1530 Max Sigma: {max_value}, Index Sigma: {max_index}')


In [ ]:
# Format data for next figure. 
data1 = MV1519_Sigma
data1.append(MV1519_LSR_Sigma[0])
data2 = MV1523_Sigma
data2.append(MV1523_LSR_Sigma[0])
data3 = MV1530_Sigma
data3.append(MV1530_LSR_Sigma[0])

Now plot each sigma as a function of training set size

Now make figures comparing the ANN parameter values to the LSR parameter values and the ANN parameter values to the ground truth values in the test set.

In [ ]:

os.chdir(r'XXXX')  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO HAVE THESE PLOTS STORED####################


font_properties = {'fontsize': 11, 'fontweight': 'normal', 'family': 'sans-serif'}

# Create a figure and axes for the subplots
fig, axs = plt.subplots(3, 1, figsize=(8, 8))

tick_labels = ['$10^{1}$ ', '$10^{2}$', '$10^{3}$' , '$10^{4}$', r'$5\times10^{4}$', '$10^{5}$', 'LSR']

# Create box plots for the first dataset
boxplot1 = axs[0].boxplot(data1,       
            vert=True, 
            patch_artist=True, 
            flierprops=dict(marker='', color='red', alpha=0.5),
            boxprops=dict(linewidth=4),
            whiskerprops=dict(linewidth=4), 
            medianprops=dict(color='black', linewidth=4))  # Vertical box plots
axs[0].set_title('(a)', fontsize=18, loc='left', y=0.85,x = 0.01)
axs[0].set_ylabel(r'$σ \times 10^{3}$', fontsize=18)
axs[0].set_ylim(-0.02, 0.22)
axs[0].set_yticks(np.arange(0.00, 0.201, 0.1))
axs[0].set_xticklabels([])
axs[0].tick_params(axis='both', which='major', labelsize=18)  # Adjust labelsize as needed
#custom_ticks = np.linspace(0.05, 0.03, 6) 
#axs[0].set_ylim(-0.005, 0.03)
#axs[0].set_yticks(custom_ticks)



# Create box plots for the second dataset
boxplot2 = axs[1].boxplot(data2,        
            vert=True, 
            patch_artist=True, 
            flierprops=dict(marker='', color='red', alpha=0.15),
            boxprops=dict(linewidth=4),
            whiskerprops=dict(linewidth=4), 
            medianprops=dict(color='black', linewidth=4))
axs[1].set_title('(b)', fontsize=18, loc='left', y=0.85,x = 0.01)
axs[1].set_ylim(-0.02, 0.22)
axs[1].set_yticks(np.arange(0.00, 0.201, 0.1))
axs[1].set_ylabel(r'$σ \times 10^{3}$', fontsize=18)
axs[1].set_xticklabels([])
axs[1].tick_params(axis='both', which='major', labelsize=18)  # Adjust labelsize as needed
#axs[1].set_ylim(-3, 4)

# Create box plots for the third dataset
boxplot3 = axs[2].boxplot(data3,    
            vert=True, 
            patch_artist=True, 
            flierprops=dict(marker='', color='red', alpha=0.5),
            boxprops=dict(linewidth=4),
            whiskerprops=dict(linewidth=4), 
            medianprops=dict(color='black', linewidth=4))
axs[2].set_title('(c)', fontsize=18, loc='left', y=0.85,x = 0.01)
axs[2].set_ylabel(r'$σ \times 10^{3}$', fontsize=18)
axs[2].set_xlabel('ANN training size', fontsize=18)
axs[2].set_xticklabels(tick_labels, fontdict=font_properties)
axs[2].tick_params(axis='both', which='major', labelsize=18, pad=5)  # Adjust labelsize as needed
axs[2].set_yticks(np.arange(0.00, 0.401, 0.2))
axs[2].set_ylim(-0.04, 0.44)

# Hide the box and whisker lines
for box in boxplot1['boxes']:
    box.set_visible(False)  # Hide the box

for whisker in boxplot1['whiskers']:
    whisker.set_visible(False)  # Hide the whiskers

# Optional: You can still display the median and fliers
for median in boxplot1['medians']:
    median.set_visible(False)  # Keep the median line visible

for flier in boxplot1['fliers']:
    flier.set_visible(False)  # Keep the outliers visible

for cap in boxplot1['caps']:
    cap.set_visible(False)  # Hide the caps

# Hide the box and whisker lines
for box in boxplot2['boxes']:
    box.set_visible(False)  # Hide the box

for whisker in boxplot2['whiskers']:
    whisker.set_visible(False)  # Hide the whiskers

# Optional: You can still display the median and fliers
for median in boxplot2['medians']:
    median.set_visible(False)  # Keep the median line visible

for flier in boxplot2['fliers']:
    flier.set_visible(False)  # Keep the outliers visible

for cap in boxplot2['caps']:
    cap.set_visible(False)  # Hide the caps


# Hide the box and whisker lines
for box in boxplot3['boxes']:
    box.set_visible(False)  # Hide the box

for whisker in boxplot3['whiskers']:
    whisker.set_visible(False)  # Hide the whiskers

# Optional: You can still display the median and fliers
for median in boxplot3['medians']:
    median.set_visible(False)  # Keep the median line visible

for flier in boxplot3['fliers']:
    flier.set_visible(False)  # Keep the outliers visible

for cap in boxplot3['caps']:
    cap.set_visible(False)  # Hide the caps


for ax in axs:
    ax.spines['top'].set_linewidth(2)
    ax.spines['right'].set_linewidth(2)
    ax.spines['left'].set_linewidth(2)
    ax.spines['bottom'].set_linewidth(2)

    # Make ticks point inward
    ax.tick_params(direction='in', width=2)



for i in range(len(data1)):
    axs[0].scatter([i + 1] * len(data1[i]), data1[i], color='red', marker='o', edgecolor = 'black')
    axs[1].scatter([i + 1] * len(data2[i]), data2[i], color='red', marker='o', edgecolor = 'black')
    axs[2].scatter([i + 1] * len(data3[i]), data3[i], color='red', marker='o', edgecolor = 'black')


#for i, label in enumerate(plt.gca().get_xticklabels()):
    #if label.get_text() == 'LSR':
        #label.set_y(label.get_position()[1] - 0.01)  # Move Group A up



def format_ticks(x, _):
    return f'{x * 1000:.1f}'  # Multiply by scale_factor

for ax in axs:
    ax.yaxis.set_major_formatter(FuncFormatter(format_ticks))


# Adjust layout
plt.tight_layout()

# Show the plot
plt.savefig("ANN_Sigma.jpg",  dpi=600, bbox_inches='tight' )
plt.show()



This next section will be making scatter plot figures comparing each predicted value to the either the ground truth values in the case of simulated data or the traditional least-squuare regression data in the case of experimental data. These plots will be generated for the versions of ANN3 and ANN4 that are trained with a set size of 50000. 

Several different error bars will be generated calculated in slightly different ways.

In [ ]:
import numpy as np
from scipy.integrate import simps
import matplotlib.patches as patches 

In [ ]:

import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

os.chdir(r'XXXX')  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO HAVE THESE PLOTS STORED####################

df1 = pd.DataFrame(ANN3_50000_test) # Store Si_ANN test predictions
df2 = pd.DataFrame(ANN4_50000_test) # Store SLG_ANN test predictions

df3 = pd.DataFrame(y_test_DS1) # Store ground truth for Data Set 1 Test Set
df4 = pd.DataFrame(y_test_DS2) # Store ground truth for Data Set 2 Test Set

############### Ep Plot ################################################
key = 'Ep'

y = pd.concat( [df1[str(key)], df2[str(key)] ], ignore_index=True) # concatenate model predictions 
x = pd.concat( [df3[str(key)], df4[str(key)] ], ignore_index=True) # concatenate ground truth values

x_line = np.linspace(min(x),max(x), 20)


# Calculate the residuals for slope = 1
residuals = y - x  # Since we want lines of the form y = x + b


data = residuals

# Step 1: Calculate the histogram
counts, bin_edges = np.histogram(data, bins=500, density=True)

# Step 2: Calculate the cumulative sum (CDF)
cumulative_counts = np.cumsum(counts)
cumulative_counts_normalized = cumulative_counts / cumulative_counts[-1]  # Normalize


# Step 3: Find the x value at 97.5% CDF
threshold1 = 0.975
x_975 = bin_edges[np.searchsorted(cumulative_counts_normalized, threshold1)]

# Step 3: Find the x value at 2.5% CDF
threshold2 = 0.025
x_025 = bin_edges[np.searchsorted(cumulative_counts_normalized, threshold2)]


# Step 3: Calculate bin centers for plotting
bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])

# Step 4: Plot the histogram and the CDF
plt.figure(figsize=(12, 6))

# Histogram
plt.subplot(1, 2, 1)
plt.bar(bin_centers, counts, width=bin_edges[1]-bin_edges[0], edgecolor='black', alpha=0.7)
plt.title('Histogram')
plt.xlabel('Value')
plt.ylabel('Density')

# CDF
plt.subplot(1, 2, 2)
plt.plot(bin_centers, cumulative_counts_normalized, marker='o')
plt.title('Cumulative Distribution Function (CDF)')
plt.xlabel('Value')
plt.ylabel('Cumulative Probability')

plt.tight_layout()
plt.show()




########### Generate Scatter Plot ###################


plt.xlim (min(x_line) , max(x_line)) # set limits on x in graph
plt.ylim(min(x_line), max(x_line) ) # set limits on Y in graph
# Calculate the upper and lower bounds for 95% of the data

lower_bound = x_025
upper_bound =  x_975

# Calculate the intercepts
intercept_lower = -lower_bound
intercept_upper = -upper_bound

l1 = plt.scatter(y_test_DS1[key], ANN3_50000_test[key], s=25,  color='blue', label='Data Set 1 Test Set')
l2 = plt.scatter(y_test_DS2[key], ANN4_50000_test[key], s=25, color='green', label='Data Set 2 Test Set' )

plt.plot(x_line, x_line, color='Black', linewidth= 3 )
#plt.plot(x_line, .95 * x_line, color='Black', linestyle='--', linewidth= 3)
#plt.plot(x_line, 1.05*x_line, color='Black', linestyle='--',  linewidth= 3)
plt.plot(x_line, x_line + intercept_lower, color='Black', linestyle='--', linewidth = 3 )
plt.plot(x_line, x_line + intercept_upper, color='Black', linestyle='--', linewidth = 3)

l3 = plt.scatter(MV1519_params[str(key)], ANN3_50000_MV1519[str(key)], s=50,  edgecolor='black', color='lime', marker='o', label='MV1519', zorder=2 )
l4 = plt.scatter(MV1523_params[str(key)], ANN3_50000_MV1523[str(key)],  s=50, edgecolor='black', color='red', marker='^', label='MV1523', zorder=2 )
l5 = plt.scatter(MV1530_params[str(key)], ANN4_50000_MV1530[str(key)], s=50, edgecolor='black', color='purple', marker='s', label='MV1530', zorder=2)

print('##### 95% Confidence ########## ')
print( key + ' ' +  str(round(upper_bound,5)))
print( key + ' ' + str(round(lower_bound,5)))

#plt.legend(fontsize=14, frameon=False, loc=(-0.015, 0.6) )
#plt.title(str(key) + ': ANN_50000')
plt.xlabel('Ground Truth $E_{p}$ (eV)', fontsize=16)
plt.ylabel('ANN $E_{p} (eV)$', fontsize=16)

first_legend = plt.legend(handles=[l1, l2], fontsize=16, frameon=False, loc=(-0.015, 0.80) )
# Manually add the first legend so it won't be overwritten


second_legend = plt.legend(handles=[l3, l4, l5], fontsize=16, frameon=False, loc=(0.6,0) )

# Plot configurations


ax = plt.gca()
ax.add_artist(first_legend)

# Set the linewidth of the axis lines
ax.spines['top'].set_linewidth(2)
ax.spines['right'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)

plt.tick_params(axis='both', which='major', width=2)  # Adjust 'width' for thickness
plt.tick_params(axis='both', which='minor', width=2)  # Optionally adjust minor ticks


ax.set_xticks([1.00, 2.00, 3.00])
ax.set_yticks([1.00, 2.00, 3.00])
# Set the tick marks to point inside
ax.tick_params(axis='both', direction='in', labelsize=16)
#ax.set_aspect('equal')



# Set y-axis and x-axis format to show 2 decimal places
ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))


plt.savefig("Ep.jpg", dpi=600, bbox_inches='tight' )  # Saves the plot as a PDF file

plt.show()
plt.close()
############### Ep Plot End ################################################



In [ ]:
df1 = pd.DataFrame(ANN3_50000_test) # Store Si_ANN test predictions
df2 = pd.DataFrame(ANN4_50000_test) # Store SLG_ANN test predictions

df3 = pd.DataFrame(y_test_DS1) # Store ground truth for Data Set 1 Test Set
df4 = pd.DataFrame(y_test_DS2) # Store ground truth for Data Set 2 Test Set

############### Eg Plot ################################################
key = 'Eg'

y = pd.concat( [df1[str(key)], df2[str(key)] ], ignore_index=True) # concatenate model predictions 
x = pd.concat( [df3[str(key)], df4[str(key)] ], ignore_index=True) # concatenate ground truth values

x_line = np.linspace(min(x),max(x), 20)
# Calculate the residuals for slope = 1
residuals = y - x  # Since we want lines of the form y = x + b

data = residuals

# Step 1: Calculate the histogram
counts, bin_edges = np.histogram(data, bins=500, density=True)

# Step 2: Calculate the cumulative sum (CDF)
cumulative_counts = np.cumsum(counts)
cumulative_counts_normalized = cumulative_counts / cumulative_counts[-1]  # Normalize


# Step 3: Find the x value at 97.5% CDF
threshold1 = 0.975
x_975 = bin_edges[np.searchsorted(cumulative_counts_normalized, threshold1)]

# Step 3: Find the x value at 2.5% CDF
threshold2 = 0.025
x_025 = bin_edges[np.searchsorted(cumulative_counts_normalized, threshold2)]


# Step 3: Calculate bin centers for plotting
bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])

# Step 4: Plot the histogram and the CDF
plt.figure(figsize=(12, 6))

# Histogram
plt.subplot(1, 2, 1)
plt.bar(bin_centers, counts, width=bin_edges[1]-bin_edges[0], edgecolor='black', alpha=0.7)
plt.title('Histogram')
plt.xlabel('Value')
plt.ylabel('Density')

# CDF
plt.subplot(1, 2, 2)
plt.plot(bin_centers, cumulative_counts_normalized, marker='o')
plt.title('Cumulative Distribution Function (CDF)')
plt.xlabel('Value')
plt.ylabel('Cumulative Probability')

plt.tight_layout()
plt.show()


########### Generate Scatter Plot ###################


plt.xlim (min(x_line) , max(x_line)) # set limits on x in graph
plt.ylim(min(x_line), max(x_line) ) # set limits on Y in graph
# Calculate the upper and lower bounds for 95% of the data
lower_bound = x_025
upper_bound =  x_975

# Calculate the intercepts
intercept_lower = -lower_bound
intercept_upper = -upper_bound

l1 = plt.scatter(y_test_DS1[key], ANN3_50000_test[key], s=25,  color='blue', label='Data Set 1 Test Set' )
l2 = plt.scatter(y_test_DS2[key], ANN4_50000_test[key], s=25, color='green', label='Data Set 2 Test Set'  )

plt.plot(x_line, x_line, color='Black', linewidth= 3 )
#plt.plot(x_line, .95 * x_line, color='Black', linestyle='--', linewidth= 3)
#plt.plot(x_line, 1.05*x_line, color='Black', linestyle='--',  linewidth= 3)
plt.plot(x_line, x_line + intercept_lower, color='Black', linestyle='--', linewidth = 3 )
plt.plot(x_line, x_line + intercept_upper, color='Black', linestyle='--', linewidth = 3)

l3 = plt.scatter(MV1519_params[str(key)], ANN3_50000_MV1519[str(key)], s=50,  edgecolor='black', color='lime', marker='o', label='MV1519', zorder=2 )
l4 = plt.scatter(MV1523_params[str(key)], ANN3_50000_MV1523[str(key)],  s=50, edgecolor='black', color='red', marker='^', label='MV1523', zorder=2 )
l5 = plt.scatter(MV1530_params[str(key)], ANN4_50000_MV1530[str(key)], s=50, edgecolor='black', color='purple', marker='s', label='MV1530', zorder=2)

print('##### 95% Confidence ########## ')
print( key + ' ' +  str(round(upper_bound,5)))
print( key + ' ' + str(round(lower_bound,5)))


#plt.title(str(key) + ': ANN_50000')
plt.xlabel('Ground Truth $E_{g}$ (eV)', fontsize=16)
plt.ylabel('ANN $E_{g}$ (eV)', fontsize=16)  


first_legend = plt.legend(handles=[l1, l2], fontsize=16, frameon=False, loc=(-0.015, 0.80) )
# Manually add the first legend so it won't be overwritten


second_legend = plt.legend(handles=[l3, l4, l5], fontsize=16, frameon=False, loc=(0.6,0) )
  
# Plot configurations


ax = plt.gca()
ax.add_artist(first_legend)

# Set the linewidth of the axis lines
ax.spines['top'].set_linewidth(2)
ax.spines['right'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)





# Set the tick marks to point inside
ax.tick_params(axis='both', direction='in', labelsize=16)

plt.tick_params(axis='both', which='major', width=2)  # Adjust 'width' for thickness
plt.tick_params(axis='both', which='minor', width=1)  # Optionally adjust minor ticks

# Get current axes
ax = plt.gca()

# Set the tick marks to point inside
ax.tick_params(axis='both', direction='in')

ax.set_xticks([1.25, 1.75, 2.25])
ax.set_yticks([1.25, 1.75, 2.25])
#ax.set_aspect('equal')

plt.savefig("Eg.jpg", dpi=600, bbox_inches='tight' )  # Saves the plot as a PDF file
plt.show()
plt.close()
############### Eg Plot End ################################################


In [ ]:
df1 = pd.DataFrame(ANN3_50000_test) # Store Si_ANN test predictions
df2 = pd.DataFrame(ANN4_50000_test) # Store SLG_ANN test predictions

df3 = pd.DataFrame(y_test_DS1) # Store ground truth for Data Set 1 Test Set
df4 = pd.DataFrame(y_test_DS2) # Store ground truth for Data Set 2 Test Set

############### Eo Plot  ################################################
key = 'Eo'

y = pd.concat( [df1[str(key)], df2[str(key)] ], ignore_index=True) # concatenate model predictions 
x = pd.concat( [df3[str(key)], df4[str(key)] ], ignore_index=True) # concatenate ground truth values

x_line = np.linspace(min(x),max(x), 20)

# Calculate the residuals for slope = 1
residuals = y - x  # Since we want lines of the form y = x + b

data = residuals

# Step 1: Calculate the histogram
counts, bin_edges = np.histogram(data, bins=500, density=True)

# Step 2: Calculate the cumulative sum (CDF)
cumulative_counts = np.cumsum(counts)
cumulative_counts_normalized = cumulative_counts / cumulative_counts[-1]  # Normalize


# Step 3: Find the x value at 97.5% CDF
threshold1 = 0.975
x_975 = bin_edges[np.searchsorted(cumulative_counts_normalized, threshold1)]

# Step 3: Find the x value at 2.5% CDF
threshold2 = 0.025
x_025 = bin_edges[np.searchsorted(cumulative_counts_normalized, threshold2)]


# Step 3: Calculate bin centers for plotting
bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])

# Step 4: Plot the histogram and the CDF
plt.figure(figsize=(12, 6))

# Histogram
plt.subplot(1, 2, 1)
plt.bar(bin_centers, counts, width=bin_edges[1]-bin_edges[0], edgecolor='black', alpha=0.7)
plt.title('Histogram')
plt.xlabel('Value')
plt.ylabel('Density')

# CDF
plt.subplot(1, 2, 2)
plt.plot(bin_centers, cumulative_counts_normalized, marker='o')
plt.title('Cumulative Distribution Function (CDF)')
plt.xlabel('Value')
plt.ylabel('Cumulative Probability')

plt.tight_layout()
plt.show()


########### Generate Scatter Plot ###################


plt.xlim (min(x_line) , max(x_line)) # set limits on x in graph
plt.ylim(min(x_line), max(x_line) ) # set limits on Y in graph
# Calculate the upper and lower bounds for 95% of the data
lower_bound = x_025
upper_bound =  x_975

# Calculate the intercepts
intercept_lower = -lower_bound
intercept_upper = -upper_bound

l1 = plt.scatter(y_test_DS1[key], ANN3_50000_test[key], s=25,  color='blue', label='Data Set 1 Test Set' )
l2 = plt.scatter(y_test_DS2[key], ANN4_50000_test[key], s=25, color='green', label='Data Set 2 Test Set'  )

plt.plot(x_line, x_line, color='Black', linewidth= 3 )
#plt.plot(x_line, .95 * x_line, color='Black', linestyle='--', linewidth= 3)
#plt.plot(x_line, 1.05*x_line, color='Black', linestyle='--',  linewidth= 3)
plt.plot(x_line, x_line + intercept_lower, color='Black', linestyle='--', linewidth= 3 )
plt.plot(x_line, x_line + intercept_upper, color='Black', linestyle='--', linewidth= 3)

l3 = plt.scatter(MV1519_params[str(key)], ANN3_50000_MV1519[str(key)], s=50,  edgecolor='black', color='lime', marker='o', label='MV1519', zorder=2 )
l4 = plt.scatter(MV1523_params[str(key)], ANN3_50000_MV1523[str(key)],  s=50, edgecolor='black', color='red', marker='^', label='MV1523', zorder=2 )
l5 = plt.scatter(MV1530_params[str(key)], ANN4_50000_MV1530[str(key)], s=50, edgecolor='black', color='purple', marker='s', label='MV1530', zorder=2)

print('##### 95% Confidence ########## ')
print( key + ' ' +  str(round(upper_bound,5)))
print( key + ' ' + str(round(lower_bound,5)))

plt.legend(fontsize=14, frameon=False, loc=(-0.015, 0.6) )
#plt.title(str(key) + ': ANN_50000')
plt.xlabel('Ground Truth $E_{0}$ (eV)', fontsize=16)
plt.ylabel('ANN $E_{0}$ (eV)', fontsize=16)

# Plot configurations

first_legend = plt.legend(handles=[l1, l2], fontsize=16, frameon=False, loc=(-0.015, 0.80) )
# Manually add the first legend so it won't be overwritten


second_legend = plt.legend(handles=[l3, l4, l5], fontsize=16, frameon=False, loc=(0.6,0) )

ax = plt.gca()
ax.add_artist(first_legend)

# Set the linewidth of the axis lines
ax.spines['top'].set_linewidth(2)
ax.spines['right'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)

plt.tick_params(axis='both', which='major', width=2, labelsize=16)  # Adjust 'width' for thickness
plt.tick_params(axis='both', which='minor', width=1, labelsize=16)  # Optionally adjust minor ticks

# Get current axes
ax = plt.gca()

# Set the tick marks to point inside
ax.tick_params(axis='both', direction='in', labelsize=16)

ax.set_xticks([3.25, 4, 4.75])
ax.set_yticks([3.25, 4, 4.75])
#ax.set_aspect('equal')

plt.savefig("Eo.jpg", dpi=600, bbox_inches='tight' )  # Saves the plot as a PDF file

plt.show()
plt.close()
############### Eo Plot End ################################################





In [ ]:
df1 = pd.DataFrame(ANN3_50000_test) # Store Si_ANN test predictions
df2 = pd.DataFrame(ANN4_50000_test) # Store SLG_ANN test predictions

df3 = pd.DataFrame(y_test_DS1) # Store ground truth for Data Set 1 Test Set
df4 = pd.DataFrame(y_test_DS2) # Store ground truth for Data Set 2 Test Set

############### Bulk Plot ################################################
key = 'BulkT'

y = pd.concat( [df1[str(key)], df2[str(key)] ], ignore_index=True) # concatenate model predictions 
x = pd.concat( [df3[str(key)], df4[str(key)] ], ignore_index=True) # concatenate ground truth values

x_line = np.linspace(min(x),max(x), 20)
# Calculate the residuals for slope = 1
residuals = y - x  # Since we want lines of the form y = x + b

data = residuals

# Step 1: Calculate the histogram
counts, bin_edges = np.histogram(data, bins=500, density=True)

# Step 2: Calculate the cumulative sum (CDF)
cumulative_counts = np.cumsum(counts)
cumulative_counts_normalized = cumulative_counts / cumulative_counts[-1]  # Normalize


# Step 3: Find the x value at 97.5% CDF
threshold1 = 0.975
x_975 = bin_edges[np.searchsorted(cumulative_counts_normalized, threshold1)]

# Step 3: Find the x value at 2.5% CDF
threshold2 = 0.025
x_025 = bin_edges[np.searchsorted(cumulative_counts_normalized, threshold2)]


# Step 3: Calculate bin centers for plotting
bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])

# Step 4: Plot the histogram and the CDF
plt.figure(figsize=(12, 6))

# Histogram
plt.subplot(1, 2, 1)
plt.bar(bin_centers, counts, width=bin_edges[1]-bin_edges[0], edgecolor='black', alpha=0.7)
plt.title('Histogram')
plt.xlabel('Value')
plt.ylabel('Density')

# CDF
plt.subplot(1, 2, 2)
plt.plot(bin_centers, cumulative_counts_normalized, marker='o')
plt.title('Cumulative Distribution Function (CDF)')
plt.xlabel('Value')
plt.ylabel('Cumulative Probability')

plt.tight_layout()
plt.show()

########### Generate Scatter Plot ###################


plt.xlim(min(x_line) , max(x_line)) # set limits on x in graph
plt.ylim(min(x_line), max(x_line) ) # set limits on Y in graph
# Calculate the upper and lower bounds for 95% of the data
lower_bound = x_025
upper_bound =  x_975

# Calculate the intercepts
intercept_lower = -lower_bound
intercept_upper = -upper_bound

l1 = plt.scatter(y_test_DS1[key], ANN3_50000_test[key], s=25,  color='blue', label='Data Set 1 Test Set' )
l2 = plt.scatter(y_test_DS2[key], ANN4_50000_test[key], s=25, color='green', label='Data Set 2 Test Set'  )

plt.plot(x_line, x_line, color='Black', linewidth= 3 )
#plt.plot(x_line, .95 * x_line, color='Black', linestyle='--', linewidth= 3)
#plt.plot(x_line, 1.05*x_line, color='Black', linestyle='--',  linewidth= 3)
plt.plot(x_line, x_line + intercept_lower, color='Black', linestyle='--', linewidth= 3 )
plt.plot(x_line, x_line + intercept_upper, color='Black', linestyle='--', linewidth= 3)

l3 = plt.scatter(MV1519_params[str(key)], ANN3_50000_MV1519[str(key)], s=50,  edgecolor='black', color='lime', marker='o', label='MV1519', zorder=2 )
l4 = plt.scatter(MV1523_params[str(key)], ANN3_50000_MV1523[str(key)],  s=50, edgecolor='black', color='red', marker='^', label='MV1523', zorder=2 )
l5 = plt.scatter(MV1530_params[str(key)], ANN4_50000_MV1530[str(key)], s=50, edgecolor='black', color='purple', marker='s', label='MV1530', zorder=2)

print('##### 95% Confidence ########## ')
print( key + ' ' +  str(round(upper_bound,5)))
print( key + ' ' + str(round(lower_bound,5)))

plt.legend(fontsize=14, frameon=False, loc=(-0.015, 0.6) )
#plt.title(str(key) + ': ANN_50000')
plt.xlabel('Ground Truth Bulk Thickness (nm)', fontsize=16)
plt.ylabel('ANN Bulk Thickness (nm)', fontsize=16)

# Plot configurations


first_legend = plt.legend(handles=[l1, l2], fontsize=16, frameon=False, loc=(-0.015, 0.80) )
# Manually add the first legend so it won't be overwritten


second_legend = plt.legend(handles=[l3, l4, l5], fontsize=16, frameon=False, loc=(0.6,0) )



ax = plt.gca()
ax.add_artist(first_legend)

# Set the linewidth of the axis lines
ax.spines['top'].set_linewidth(2)
ax.spines['right'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)

plt.tick_params(axis='both', which='major', width=2)  # Adjust 'width' for thickness
plt.tick_params(axis='both', which='minor', width=1)  # Optionally adjust minor ticks

# Get current axes
ax = plt.gca()

# Set the tick marks to point inside
ax.tick_params(axis='both', direction='in', labelsize=16)

ax.set_xticks([50, 75, 100])
ax.set_yticks([50, 75, 100])
#ax.set_aspect('equal')


plt.savefig("BulkT.jpg", dpi=600, bbox_inches='tight' )  # Saves the plot as a PDF file
plt.show()
plt.close()
############### BulkT Plot End ################################################





In [ ]:
df1 = pd.DataFrame(ANN3_50000_test) # Store Si_ANN test predictions
df2 = pd.DataFrame(ANN4_50000_test) # Store SLG_ANN test predictions

df3 = pd.DataFrame(y_test_DS1) # Store ground truth for Data Set 1 Test Set
df4 = pd.DataFrame(y_test_DS2) # Store ground truth for Data Set 2 Test Set

############### Br Plot ################################################

key = 'Br'

y = pd.concat( [df1[str(key)], df2[str(key)] ], ignore_index=True) # concatenate model predictions 
x = pd.concat( [df3[str(key)], df4[str(key)] ], ignore_index=True) # concatenate ground truth values

x_line = np.linspace(min(x),max(x), 20)

# Calculate the residuals for slope = 1
residuals = y - x  # Since we want lines of the form y = x + b

data = residuals

# Step 1: Calculate the histogram
counts, bin_edges = np.histogram(data, bins=500, density=True)

# Step 2: Calculate the cumulative sum (CDF)
cumulative_counts = np.cumsum(counts)
cumulative_counts_normalized = cumulative_counts / cumulative_counts[-1]  # Normalize


# Step 3: Find the x value at 97.5% CDF
threshold1 = 0.975
x_975 = bin_edges[np.searchsorted(cumulative_counts_normalized, threshold1)]

# Step 3: Find the x value at 2.5% CDF
threshold2 = 0.025
x_025 = bin_edges[np.searchsorted(cumulative_counts_normalized, threshold2)]


# Step 3: Calculate bin centers for plotting
bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])

# Step 4: Plot the histogram and the CDF
plt.figure(figsize=(12, 6))

# Histogram
plt.subplot(1, 2, 1)
plt.bar(bin_centers, counts, width=bin_edges[1]-bin_edges[0], edgecolor='black', alpha=0.7)
plt.title('Histogram')
plt.xlabel('Value')
plt.ylabel('Density')

# CDF
plt.subplot(1, 2, 2)
plt.plot(bin_centers, cumulative_counts_normalized, marker='o')
plt.title('Cumulative Distribution Function (CDF)')
plt.xlabel('Value')
plt.ylabel('Cumulative Probability')

plt.tight_layout()
plt.show()



########### Generate Scatter Plot ###################


plt.xlim(min(x_line) , max(x_line)) # set limits on x in graph
plt.ylim(min(x_line), max(x_line) ) # set limits on Y in graph
# Calculate the upper and lower bounds for 95% of the data
lower_bound = x_025
upper_bound =  x_975

# Calculate the intercepts
intercept_lower = -lower_bound
intercept_upper = -upper_bound

l1 = plt.scatter(y_test_DS1[key], ANN3_50000_test[key], s=25,  color='blue', label='Data Set 1 Test Set' )
l2 = plt.scatter(y_test_DS2[key], ANN4_50000_test[key], s=25, color='green', label='Data Set 2 Test Set'  )

plt.plot(x_line, x_line, color='Black', linewidth= 3 )
#plt.plot(x_line, .95 * x_line, color='Black', linestyle='--', linewidth= 3)
#plt.plot(x_line, 1.05*x_line, color='Black', linestyle='--',  linewidth= 3)
plt.plot(x_line, x_line + intercept_lower, color='Black', linestyle='--', linewidth= 3 )
plt.plot(x_line, x_line + intercept_upper, color='Black', linestyle='--', linewidth= 3)

l3 = plt.scatter(MV1519_params[str(key)], ANN3_50000_MV1519[str(key)], s=50,  edgecolor='black', color='lime', marker='o', label='MV1519', zorder=2 )
l4 = plt.scatter(MV1523_params[str(key)], ANN3_50000_MV1523[str(key)],  s=50, edgecolor='black', color='red', marker='^', label='MV1523', zorder=2 )
l5 = plt.scatter(MV1530_params[str(key)], ANN4_50000_MV1530[str(key)], s=50, edgecolor='black', color='purple', marker='s', label='MV1530', zorder=2)

print('##### 95% Confidence ########## ')
print( key + ' ' +  str(round(upper_bound,5)))
print( key + ' ' + str(round(lower_bound,5)))

plt.legend(fontsize=14, frameon=False, loc=(-0.015, 0.6) )
#plt.title(str(key) + ': ANN_50000')
plt.xlabel('Ground Truth ' + r'$\Gamma$ (eV)',fontsize=16)
plt.ylabel('ANN ' + r'$\Gamma$ (eV)',fontsize=16)
    
# Plot configurations
first_legend = plt.legend(handles=[l1, l2], fontsize=16, frameon=False, loc=(-0.015, 0.80) )
# Manually add the first legend so it won't be overwritten


second_legend = plt.legend(handles=[l3, l4, l5], fontsize=16, frameon=False, loc=(0.6,0) )


ax = plt.gca()
ax.add_artist(first_legend)



# Set the linewidth of the axis lines
ax.spines['top'].set_linewidth(2)
ax.spines['right'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)

plt.tick_params(axis='both', which='major', width=2)  # Adjust 'width' for thickness
plt.tick_params(axis='both', which='minor', width=1)  # Optionally adjust minor ticks

# Get current axes
ax = plt.gca()

# Set the tick marks to point inside
ax.tick_params(axis='both', direction='in', labelsize=16)

ax.set_xticks([1.75, 2.5, 3.25])
ax.set_yticks([1.75, 2.5, 3.25])
#ax.set_aspect('equal')

plt.savefig("Br.jpg", dpi=600, bbox_inches='tight' )  # Saves the plot as a PDF file

plt.show()
plt.close()
############### Br Plot End ################################################






In [ ]:
df1 = pd.DataFrame(ANN3_50000_test) # Store Si_ANN test predictions
df2 = pd.DataFrame(ANN4_50000_test) # Store SLG_ANN test predictions

df3 = pd.DataFrame(y_test_DS1) # Store ground truth for Data Set 1 Test Set
df4 = pd.DataFrame(y_test_DS2) # Store ground truth for Data Set 2 Test Set

############### Br Plot ################################################

key = 'Amp'

y = pd.concat( [df1[str(key)], df2[str(key)] ], ignore_index=True) # concatenate model predictions 
x = pd.concat( [df3[str(key)], df4[str(key)] ], ignore_index=True) # concatenate ground truth values

x_line = np.linspace(min(x),max(x), 20)

# Calculate the residuals for slope = 1
residuals = y - x  # Since we want lines of the form y = x + b

data = residuals

# Step 1: Calculate the histogram
counts, bin_edges = np.histogram(data, bins=500, density=True)

# Step 2: Calculate the cumulative sum (CDF)
cumulative_counts = np.cumsum(counts)
cumulative_counts_normalized = cumulative_counts / cumulative_counts[-1]  # Normalize


# Step 3: Find the x value at 97.5% CDF
threshold1 = 0.975
x_975 = bin_edges[np.searchsorted(cumulative_counts_normalized, threshold1)]

# Step 3: Find the x value at 2.5% CDF
threshold2 = 0.025
x_025 = bin_edges[np.searchsorted(cumulative_counts_normalized, threshold2)]


# Step 3: Calculate bin centers for plotting
bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])

# Step 4: Plot the histogram and the CDF
plt.figure(figsize=(12, 6))

# Histogram
plt.subplot(1, 2, 1)
plt.bar(bin_centers, counts, width=bin_edges[1]-bin_edges[0], edgecolor='black', alpha=0.7)
plt.title('Histogram')
plt.xlabel('Value')
plt.ylabel('Density')

# CDF
plt.subplot(1, 2, 2)
plt.plot(bin_centers, cumulative_counts_normalized, marker='o')
plt.title('Cumulative Distribution Function (CDF)')
plt.xlabel('Value')
plt.ylabel('Cumulative Probability')

plt.tight_layout()
plt.show()

########### Generate Scatter Plot ###################


plt.xlim(min(x_line) , max(x_line)) # set limits on x in graph
plt.ylim(min(x_line), max(x_line) ) # set limits on Y in graph
# Calculate the upper and lower bounds for 95% of the data
lower_bound = x_025
upper_bound =  x_975

# Calculate the intercepts
intercept_lower = -lower_bound
intercept_upper = -upper_bound

l1 = plt.scatter(y_test_DS1[key], ANN3_50000_test[key], s=25,  color='blue', label='Data Set 1 Test Set' )
l2 = plt.scatter(y_test_DS2[key], ANN4_50000_test[key], s=25, color='green', label='Data Set 2 Test Set'  )

plt.plot(x_line, x_line, color='Black', linewidth= 3 )
#plt.plot(x_line, .95 * x_line, color='Black', linestyle='--', linewidth= 3)
#plt.plot(x_line, 1.05*x_line, color='Black', linestyle='--',  linewidth= 3)
plt.plot(x_line, x_line + intercept_lower, color='Black', linestyle='--', linewidth= 3 )
plt.plot(x_line, x_line + intercept_upper, color='Black', linestyle='--', linewidth= 3)

l3 = plt.scatter(MV1519_params[str(key)], ANN3_50000_MV1519[str(key)], s=50,  edgecolor='black', color='lime', marker='o', label='MV1519', zorder=2 )
l4 = plt.scatter(MV1523_params[str(key)], ANN3_50000_MV1523[str(key)],  s=50, edgecolor='black', color='red', marker='^', label='MV1523', zorder=2 )
l5 = plt.scatter(MV1530_params[str(key)], ANN4_50000_MV1530[str(key)], s=50, edgecolor='black', color='purple', marker='s', label='MV1530', zorder=2)



print('##### 95% Confidence ########## ')
print( key + ' ' +  str(round(upper_bound,5)))
print( key + ' ' + str(round(lower_bound,5)))

plt.legend(fontsize=14, frameon=False, loc=(-0.015, 0.6) )
#plt.title(str(key) + ': ANN_50000')
plt.xlabel('Ground Truth $A$ (eV)', fontsize=16)
plt.ylabel('ANN $A$ (eV)', fontsize=16)

first_legend = plt.legend(handles=[l1, l2], fontsize=16, frameon=False, loc=(-0.015, 0.80) )
# Manually add the first legend so it won't be overwritten


second_legend = plt.legend(handles=[l3, l4, l5], fontsize=16, frameon=False, loc=(0.6,0) )

    
# Plot configurations


ax = plt.gca()
ax.add_artist(first_legend)


# Set the linewidth of the axis lines
ax.spines['top'].set_linewidth(2)
ax.spines['right'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)

plt.tick_params(axis='both', which='major', width=2)  # Adjust 'width' for thickness
plt.tick_params(axis='both', which='minor', width=1)  # Optionally adjust minor ticks

# Get current axes
ax = plt.gca()

# Set the tick marks to point inside
ax.tick_params(axis='both', direction='in', labelsize =16)

ax.set_xticks([60, 100, 140])
ax.set_yticks([60, 100, 140])
#ax.set_aspect('equal')

plt.savefig("Amp.jpg", dpi=600, bbox_inches='tight' )  # Saves the plot as a PDF file

plt.show()
plt.close()
############### Amp Plot End ################################################





In [ ]:
df2 = pd.DataFrame(ANN4_50000_test) # Store SLG_ANN test predictions
df4 = pd.DataFrame(y_test_DS2) # Store ground truth for Data Set 2 Test Set

############### EMAT Plot ################################################
key = 'EMAT'

y = df2[str(key)]
x = df4[str(key)]

x_line = np.linspace(min(x),max(x), 20)
residuals = y - x  # Since we want lines of the form y = x + b

data = residuals

# Step 1: Calculate the histogram
counts, bin_edges = np.histogram(data, bins=500, density=True)

# Step 2: Calculate the cumulative sum (CDF)
cumulative_counts = np.cumsum(counts)
cumulative_counts_normalized = cumulative_counts / cumulative_counts[-1]  # Normalize


# Step 3: Find the x value at 97.5% CDF
threshold1 = 0.975
x_975 = bin_edges[np.searchsorted(cumulative_counts_normalized, threshold1)]

# Step 3: Find the x value at 2.5% CDF
threshold2 = 0.025
x_025 = bin_edges[np.searchsorted(cumulative_counts_normalized, threshold2)]


# Step 3: Calculate bin centers for plotting
bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])

# Step 4: Plot the histogram and the CDF
plt.figure(figsize=(12, 6))

# Histogram
plt.subplot(1, 2, 1)
plt.bar(bin_centers, counts, width=bin_edges[1]-bin_edges[0], edgecolor='black', alpha=0.7)
plt.title('Histogram')
plt.xlabel('Value')
plt.ylabel('Density')

# CDF
plt.subplot(1, 2, 2)
plt.plot(bin_centers, cumulative_counts_normalized, marker='o')
plt.title('Cumulative Distribution Function (CDF)')
plt.xlabel('Value')
plt.ylabel('Cumulative Probability')

plt.tight_layout()
plt.show()

########### Generate Scatter Plot ###################

plt.xlim(min(x_line) , max(x_line)) # set limits on x in graph
plt.ylim(min(x_line), max(x_line) ) # set limits on Y in graph
# Calculate the upper and lower bounds for 95% of the data
lower_bound = x_025
upper_bound =  x_975

# Calculate the intercepts
intercept_lower = -lower_bound
intercept_upper = -upper_bound

l1 = plt.scatter(y_test_DS2[key], ANN4_50000_test[key], s=25, color='green', label='Data Set 2 Test Set'  )

plt.plot(x_line, x_line, color='Black', linewidth= 3 )
#plt.plot(x_line, .95 * x_line, color='Black', linestyle='--', linewidth= 3)
#plt.plot(x_line, 1.05*x_line, color='Black', linestyle='--',  linewidth= 3)
l2 = plt.plot(x_line, x_line + intercept_lower, color='Black', linestyle='--', linewidth= 3 )
l3 = plt.plot(x_line, x_line + intercept_upper, color='Black', linestyle='--', linewidth= 3)

plt.scatter(MV1530_params[str(key)], ANN4_50000_MV1530[str(key)], s=50, edgecolor='black', color='purple', marker='s', label='MV1530', zorder=2)

print('##### 95% Confidence ########## ')
print( key + ' ' +  str(round(upper_bound,5)))
print( key + ' ' + str(round(lower_bound,5)))

plt.legend(fontsize=16, frameon=False, loc=(-0.015, 0.8) )
#plt.title(str(key) + ': ANN_50000')
plt.xlabel('Ground Truth Surface Layer Thickness (nm)', fontsize=16)
plt.ylabel('ANN Surface Layer Thickness (nm)', fontsize=16)

    
# Plot configurations


ax = plt.gca()

# Set the linewidth of the axis lines
ax.spines['top'].set_linewidth(2)
ax.spines['right'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)

plt.tick_params(axis='both', which='major', width=2)  # Adjust 'width' for thickness
plt.tick_params(axis='both', which='minor', width=1)  # Optionally adjust minor ticks

# Get current axes
ax = plt.gca()

# Set the tick marks to point inside
ax.tick_params(axis='both', direction='in', labelsize=16)

ax.set_xticks([0.25, 1, 1.75])
ax.set_yticks([0.25, 1, 1.75])
#ax.set_aspect('equal')

plt.savefig("EMAT.jpg", dpi=600, bbox_inches='tight' )  # Saves the plot as a PDF file

plt.show()
plt.close()
############### EMAT Plot End ################################################




For the final exercises, the values predicted by ANN3 and ANN4 will be used as input for a least-square regression in python to confirm that the ANN predictions can lead to a good fit. 

Now a function that can take predictions from ANN3 and perfrom least-square regression will be defined as well as a least-square procedure.

In [ ]:
# The least-square regression will be done using python curve-fit

# First an LSR will be built for handeling the different structural models 

def LSR_ANN3_model_func(x, Ep, Eg, Eo, Br, Amp, BulkT):

    # Step 1: Take Parameters and make the CL material
    CL = Get_CL_Material(E, Ep, Eg, Eo, Br, Amp, Eg, 1, wv) # make sure "E" and "wv" are defined before running this function
    
    # Step 2: Get Bulk Thickness
    Bulk_Thickness = BulkT
    NTVE_Thickness = NTVE_T # make sure to define NTVE_T before running
    
    # Step 3: Define Structure
    Structure = [Void,  CL, NTVE_JAW, Si_JAW]
    
    # Step 4: Creat ellipsometric spectra 
    Mat_Thick = np.array( [ Bulk_Thickness, NTVE_Thickness])
    Theta_Incident = 64.93
    df = SE_Sim(Structure, Theta_Incident, Mat_Thick, write_data=False, NCS=True)
    df = df.drop(df.columns[0], axis=1)
    data = df.to_numpy()
    data = data.ravel(order='F') # lines up data in format [N1,N2,N3....C1,C2,C3....S1,S2,S3....]
    data = data.tolist()
    
    return(data) # store prediction in the list for that model

def LSR_ANN3(exp_data, initial_params):

    x_data = np.concatenate((E, E, E))
    y_data = exp_data.ravel(order='F')

    lower_limit = 0.01
    upper_limit = 1000

    lower_bound =  np.full(initial_params.shape, lower_limit)
    upper_bound =  np.full(initial_params.shape, upper_limit)
     
    params, covariance = curve_fit( LSR_ANN3_model_func, x_data, y_data, p0=initial_params, ftol=1e-3, xtol=1e-3, gtol=1e-3, bounds=(lower_bound, upper_bound) )
    
    print(params)
    
    return(params)






def LSR_ANN4_model_func(x, Ep, Eg, Eo, Br, Amp, BulkT, EMAT):

    # Step 1: Take Parameters and make the CL material
    CL = Get_CL_Material(E, Ep, Eg, Eo, Br, Amp, Eg, 1, wv) # make sure "E" and "wv" are defined before running this function
    Surface_Layer = Bruggeman_EMA( CL, Void , 0.5 )
    SLG_Interface = Bruggeman_EMA( CL, SLG , 0.5 )
    
    
    # Step 2: Get Bulk Thickness
    Bulk_Thickness = BulkT
    Surface_Layer_Thickness = EMAT
    SLG_Interface_Thickness = 3
    
    # Step 3: Define Structure
    Structure = [Void, Surface_Layer, CL, SLG_Interface, SLG]
    
    # Step 4: Creat ellipsometric spectra 
    Mat_Thick = np.array( [  Surface_Layer_Thickness, Bulk_Thickness, SLG_Interface_Thickness])
    Theta_Incident = 64.93
    df = SE_Sim(Structure, Theta_Incident, Mat_Thick, write_data=False, NCS=True)
    df = df.drop(df.columns[0], axis=1)
    data = df.to_numpy()
    data = data.ravel(order='F') # lines up data in format [N1,N2,N3....C1,C2,C3....S1,S2,S3....]
    data = data.tolist()
    
    return(data) # store prediction in the list for that model



def LSR_ANN4(exp_data, initial_params):

    x_data = np.concatenate((E, E, E))
    y_data = exp_data.ravel(order='F')

    lower_limit = 0.01
    upper_limit = 1000

    lower_bound =  np.full(initial_params.shape, lower_limit)
    upper_bound =  np.full(initial_params.shape, upper_limit)
     
    params, covariance = curve_fit( LSR_ANN4_model_func, x_data, y_data, p0=initial_params, ftol=1e-3, xtol=1e-3, gtol=1e-3, bounds=(lower_bound, upper_bound) )
    
    print(params)
    
    return(params)





In [ ]:
# Time to test the LSR functions 

NTVE_T = 1.68
# Take predictions from ANN
initial_params = np.array([ # initial parameters from ANNN3_100000 for 1st point in MV1519 map.
     ANN3_100000_MV1519['Ep'][0], 
     ANN3_100000_MV1519['Eg'][0], 
     ANN3_100000_MV1519['Eo'][0], 
     ANN3_100000_MV1519['Br'][0],
     ANN3_100000_MV1519['Amp'][0],
     ANN3_100000_MV1519['BulkT'][0],
     ])

exp_data = MV1519[0][:, 1:] # 1st point on sample MV1519 map

params = LSR_ANN3(exp_data, initial_params)


In [ ]:
# Now to make one more set of functions to time the LSR. The time can be used to estimate performance. 

def LSR_ANN3_Timed(ANN3_predictions, experimental_data): 

    LSR_Params = [] # store final parameters after the LSR
    LSR_Times = [] # store the times it takes to fit each point

    for i in range(len(ANN3_predictions['Ep'])): # iteratre over every prediction

        # Take predictions from ANN
        initial_params = np.array([
             ANN3_predictions['Ep'][i], 
             ANN3_predictions['Eg'][i], 
             ANN3_predictions['Eo'][i], 
             ANN3_predictions['Br'][i],
             ANN3_predictions['Amp'][i],
             ANN3_predictions['BulkT'][i],
             ])
        
        timestamp1 = time.time() # start timer
         # Perform least-square regression
        params = LSR_ANN3(experimental_data[i][:, 1:] , initial_params )
        timestamp2 = time.time() # end timer

        iteration_time = timestamp2 - timestamp1 # calculate time delta
        print("Iteration time: " + str(iteration_time)) # print time delta

        LSR_Params.append(params)  # save parameters 
        LSR_Times.append(iteration_time)  # save parameters 

    return(LSR_Params, LSR_Times)

    

In [ ]:
# LSR for ANN3 trained on 100000 for MV1519
NTVE_T = 1.68
ANN3_100000_MV1519_LSR_params, ANN3_100000_MV1519_LSR_times = LSR_ANN3_Timed(ANN3_100000_MV1519, MV1519)

In [ ]:
# LSR for ANN3 trained on 50000 for MV1519
NTVE_T = 1.68
ANN3_50000_MV1519_LSR_params, ANN3_50000_MV1519_LSR_times = LSR_ANN3_Timed(ANN3_50000_MV1519, MV1519)

In [ ]:
# LSR for ANN3 trained on 10000 for MV1519
NTVE_T = 1.68
ANN3_10000_MV1519_LSR_params, ANN3_10000_MV1519_LSR_times = LSR_ANN3_Timed(ANN3_10000_MV1519, MV1519)

In [ ]:
# LSR for ANN3 trained on 1000 for MV1519
NTVE_T = 1.68
ANN3_1000_MV1519_LSR_params, ANN3_1000_MV1519_LSR_times = LSR_ANN3_Timed(ANN3_1000_MV1519, MV1519)

In [ ]:
# LSR for ANN3 trained on 100 for MV1519
NTVE_T = 1.68
ANN3_100_MV1519_LSR_params, ANN3_100_MV1519_LSR_times = LSR_ANN3_Timed(ANN3_100_MV1519, MV1519)

In [ ]:
# LSR for ANN3 trained on 10 for MV1519
NTVE_T = 1.68
ANN3_10_MV1519_LSR_params, ANN3_10_MV1519_LSR_times = LSR_ANN3_Timed(ANN3_10_MV1519, MV1519)

In [ ]:
# LSR for ANN3 trained on 100000 for MV1523
NTVE_T = 1.65
ANN3_100000_MV1523_LSR_params, ANN3_100000_MV1523_LSR_times = LSR_ANN3_Timed(ANN3_100000_MV1523, MV1523)

In [ ]:
# LSR for ANN3 trained on 50000 for MV1523
NTVE_T = 1.65
ANN3_50000_MV1523_LSR_params, ANN3_50000_MV1523_LSR_times = LSR_ANN3_Timed(ANN3_50000_MV1523, MV1523)

In [ ]:
# LSR for ANN3 trained on 10000 for MV1523
NTVE_T = 1.65
ANN3_10000_MV1523_LSR_params, ANN3_10000_MV1523_LSR_times = LSR_ANN3_Timed(ANN3_10000_MV1523, MV1523)

In [ ]:
# LSR for ANN3 trained on 1000 for MV1523
NTVE_T = 1.65
ANN3_1000_MV1523_LSR_params, ANN3_1000_MV1523_LSR_times = LSR_ANN3_Timed(ANN3_1000_MV1523, MV1523)

In [ ]:
# LSR for ANN3 trained on 100 for MV1523
NTVE_T = 1.65
ANN3_100_MV1523_LSR_params, ANN3_100_MV1523_LSR_times = LSR_ANN3_Timed(ANN3_100_MV1523, MV1523)

In [ ]:
# LSR for ANN3 trained on 10 for MV1523
NTVE_T = 1.65
ANN3_10_MV1523_LSR_params, ANN3_10_MV1523_LSR_times = LSR_ANN3_Timed(ANN3_10_MV1523, MV1523)

Now a function that can take predictions from ANN4 and perfrom least-square regression will be defined

In [ ]:

def LSR_ANN4_Timed(ANN4_predictions, experimental_data): 

    LSR_Params = [] # store final parameters after the LSR
    LSR_Times = [] # store the times it takes to fit each point

    for i in range(len(ANN4_predictions['Ep'])): # iteratre over every prediction

        # Take predictions from ANN
        initial_params = np.array([
             ANN4_predictions['Ep'][i], 
             ANN4_predictions['Eg'][i], 
             ANN4_predictions['Eo'][i], 
             ANN4_predictions['Br'][i],
             ANN4_predictions['Amp'][i],
             ANN4_predictions['BulkT'][i],
             ANN4_predictions['EMAT'][i],
             ])
        
        timestamp1 = time.time() # start timer
         # Perform least-square regression
        params = LSR_ANN4(experimental_data[i][:, 1:] , initial_params )
        timestamp2 = time.time() # end timer

        iteration_time = timestamp2 - timestamp1 # calculate time delta
        print("Iteration time: " + str(iteration_time)) # print time delta

        LSR_Params.append(params)  # save parameters 
        LSR_Times.append(iteration_time)  # save parameters 

    return(LSR_Params, LSR_Times)



In [ ]:
# LSR for ANN4 trained on 100000 for MV1530
ANN4_100000_MV1530_LSR_params, ANN4_100000_MV1530_LSR_times = LSR_ANN4_Timed(ANN4_100000_MV1530, MV1530)

In [ ]:
# LSR for ANN4 trained on 50000 for MV1530
ANN4_50000_MV1530_LSR_params, ANN4_50000_MV1530_LSR_times =  LSR_ANN4_Timed(ANN4_50000_MV1530, MV1530)

In [ ]:
# LSR for ANN4 trained on 10000 for MV1530
ANN4_10000_MV1530_LSR_params, ANN4_10000_MV1530_LSR_times = LSR_ANN4_Timed(ANN4_10000_MV1530, MV1530)

In [ ]:
# LSR for ANN4 trained on 1000 for MV1530
ANN4_1000_MV1530_LSR_params, ANN4_1000_MV1530_LSR_times = LSR_ANN4_Timed(ANN4_1000_MV1530, MV1530)

In [ ]:
# LSR for ANN4 trained on 100 for MV1530
ANN4_100_MV1530_LSR_params, ANN4_100_MV1530_LSR_times = LSR_ANN4_Timed(ANN4_100_MV1530, MV1530)

In [ ]:
# LSR for ANN4 trained on 10 for MV1530
ANN4_10_MV1530_LSR_params, ANN4_10_MV1530_LSR_times = LSR_ANN4_Timed(ANN4_10_MV1530, MV1530)

Now all the data from this least-square regression process can be stored

In [ ]:
# Store data here from MV1530
os.chdir(r'XXXX')  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO HAVE THE ANN + LSR DATA STORED####################

columns = [ 'Ep', 'Eg', 'Eo', 'Br', 'Amp', 'BulkT', 'EMAT']

####### Store Parameters from MV1530 analysis ###########
ANN4_10_MV1530_LSR  = pd.DataFrame(ANN4_10_MV1530_LSR_params, columns = columns) 
ANN4_10_MV1530_LSR.to_csv('ANN4_10_MV1530_LSR_params.csv')

ANN4_100_MV1530_LSR = pd.DataFrame(ANN4_100_MV1530_LSR_params, columns = columns) 
ANN4_100_MV1530_LSR.to_csv('ANN4_100_MV1530_LSR_params.csv')

ANN4_1000_MV1530_LSR = pd.DataFrame(ANN4_1000_MV1530_LSR_params, columns = columns) 
ANN4_1000_MV1530_LSR.to_csv('ANN4_1000_MV1530_LSR_params.csv')

ANN4_10000_MV1530_LSR = pd.DataFrame(ANN4_10000_MV1530_LSR_params, columns = columns) 
ANN4_10000_MV1530_LSR.to_csv('ANN4_10000_MV1530_LSR_params.csv')

ANN4_50000_MV1530_LSR = pd.DataFrame(ANN4_50000_MV1530_LSR_params, columns = columns) 
ANN4_50000_MV1530_LSR.to_csv('ANN4_50000_MV1530_LSR_params.csv')

ANN4_100000_MV1530_LSR = pd.DataFrame(ANN4_100000_MV1530_LSR_params, columns = columns) 
ANN4_100000_MV1530_LSR.to_csv('ANN4_100000_MV1530_LSR_params.csv')

####### Store times from MV1530 analysis ###########
df = pd.DataFrame(ANN4_10_MV1530_LSR_times) 
df.to_csv('ANN4_10_MV1530_LSR_times.csv')

df = pd.DataFrame(ANN4_100_MV1530_LSR_times) 
df.to_csv('ANN4_100_MV1530_LSR_times.csv')

df = pd.DataFrame(ANN4_1000_MV1530_LSR_times) 
df.to_csv('ANN4_1000_MV1530_LSR_times.csv')

df = pd.DataFrame(ANN4_10000_MV1530_LSR_times) 
df.to_csv('ANN4_10000_MV1530_LSR_times.csv')

df = pd.DataFrame(ANN4_50000_MV1530_LSR_times) 
df.to_csv('ANN4_50000_MV1530_LSR_times.csv')

df = pd.DataFrame(ANN4_100000_MV1530_LSR_times) 
df.to_csv('ANN4_100000_MV1530_LSR_times.csv')



In [ ]:
# Store data here from MV1519

os.chdir(r'XXXX')  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO HAVE THE ANN + LSR DATA STORED####################

columns = [ 'Ep', 'Eg', 'Eo', 'Br', 'Amp', 'BulkT']

####### Store Parameters from MV1519 analysis ###########
ANN3_10_MV1519_LSR = pd.DataFrame(ANN3_10_MV1519_LSR_params, columns = columns) 
ANN3_10_MV1519_LSR.to_csv('ANN3_10_MV1519_LSR_params.csv')

ANN3_100_MV1519_LSR = pd.DataFrame(ANN3_100_MV1519_LSR_params, columns = columns) 
ANN3_100_MV1519_LSR.to_csv('ANN3_100_MV1519_LSR_params.csv')

ANN3_1000_MV1519_LSR = pd.DataFrame(ANN3_1000_MV1519_LSR_params, columns = columns) 
ANN3_1000_MV1519_LSR.to_csv('ANN3_1000_MV1519_LSR_params.csv')

ANN3_10000_MV1519_LSR = pd.DataFrame(ANN3_10000_MV1519_LSR_params, columns = columns) 
ANN3_10000_MV1519_LSR.to_csv('ANN3_10000_MV1519_LSR_params.csv')

ANN3_50000_MV1519_LSR = pd.DataFrame(ANN3_50000_MV1519_LSR_params, columns = columns) 
ANN3_50000_MV1519_LSR.to_csv('ANN3_50000_MV1519_LSR_params.csv')

ANN3_100000_MV1519_LSR = pd.DataFrame(ANN3_100000_MV1519_LSR_params, columns = columns) 
ANN3_100000_MV1519_LSR.to_csv('ANN3_100000_MV1519_LSR_params.csv')

####### Store times from MV1519 analysis ###########
df = pd.DataFrame(ANN3_10_MV1519_LSR_times) 
df.to_csv('ANN3_10_MV1519_LSR_times.csv')

df = pd.DataFrame(ANN3_100_MV1519_LSR_times) 
df.to_csv('ANN3_100_MV1519_LSR_times.csv')

df = pd.DataFrame(ANN3_1000_MV1519_LSR_times) 
df.to_csv('ANN3_1000_MV1519_LSR_times.csv')

df = pd.DataFrame(ANN3_10000_MV1519_LSR_times) 
df.to_csv('ANN3_10000_MV1519_LSR_times.csv')

df = pd.DataFrame(ANN3_50000_MV1519_LSR_times) 
df.to_csv('ANN3_50000_MV1519_LSR_times.csv')

df = pd.DataFrame(ANN3_100000_MV1519_LSR_times) 
df.to_csv('ANN3_100000_MV1519_LSR_times.csv')



In [ ]:
# Store data here from MV1523 

os.chdir(r'XXXX')  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO HAVE THE ANN + LSR DATA STORED####################

columns = [ 'Ep', 'Eg', 'Eo', 'Br', 'Amp', 'BulkT']

####### Store Parameters from MV1523 analysis ###########
ANN3_10_MV1523_LSR = pd.DataFrame(ANN3_10_MV1523_LSR_params, columns = columns) 
ANN3_10_MV1523_LSR.to_csv('ANN3_10_MV1523_LSR_params.csv')

ANN3_100_MV1523_LSR = pd.DataFrame(ANN3_100_MV1523_LSR_params, columns = columns) 
ANN3_100_MV1523_LSR.to_csv('ANN3_100_MV1523_LSR_params.csv')

ANN3_1000_MV1523_LSR = pd.DataFrame(ANN3_1000_MV1523_LSR_params, columns = columns) 
ANN3_1000_MV1523_LSR.to_csv('ANN3_1000_MV1523_LSR_params.csv')

ANN3_10000_MV1523_LSR = pd.DataFrame(ANN3_10000_MV1523_LSR_params, columns = columns) 
ANN3_10000_MV1523_LSR.to_csv('ANN3_10000_MV1523_LSR_params.csv')

ANN3_50000_MV1523_LSR = pd.DataFrame(ANN3_50000_MV1523_LSR_params, columns = columns) 
ANN3_50000_MV1523_LSR.to_csv('ANN3_50000_MV1523_LSR_params.csv')

ANN3_100000_MV1523_LSR = pd.DataFrame(ANN3_100000_MV1523_LSR_params, columns = columns) 
ANN3_100000_MV1523_LSR.to_csv('ANN3_100000_MV1523_LSR_params.csv')

####### Store times from MV1523 analysis ###########
df = pd.DataFrame(ANN3_10_MV1523_LSR_times) 
df.to_csv('ANN3_10_MV1523_LSR_times.csv')

df = pd.DataFrame(ANN3_100_MV1523_LSR_times) 
df.to_csv('ANN3_100_MV1523_LSR_times.csv')

df = pd.DataFrame(ANN3_1000_MV1523_LSR_times) 
df.to_csv('ANN3_1000_MV1523_LSR_times.csv')

df = pd.DataFrame(ANN3_10000_MV1523_LSR_times) 
df.to_csv('ANN3_10000_MV1523_LSR_times.csv')

df = pd.DataFrame(ANN3_50000_MV1523_LSR_times) 
df.to_csv('ANN3_50000_MV1523_LSR_times.csv')

df = pd.DataFrame(ANN3_100000_MV1523_LSR_times) 
df.to_csv('ANN3_100000_MV1523_LSR_times.csv')



In [ ]:
# Now calculate the total time spent iterating each sample

Total_Time_MV1519 = [
    sum(ANN3_100000_MV1519_LSR_times),
    sum(ANN3_50000_MV1519_LSR_times),
    sum(ANN3_10000_MV1519_LSR_times),
    sum(ANN3_1000_MV1519_LSR_times),
    sum(ANN3_100_MV1519_LSR_times),
    sum(ANN3_10_MV1519_LSR_times)
]

Total_Time_MV1523 = [
    sum(ANN3_100000_MV1523_LSR_times),
    sum(ANN3_50000_MV1523_LSR_times),
    sum(ANN3_10000_MV1523_LSR_times),
    sum(ANN3_1000_MV1523_LSR_times),
    sum(ANN3_100_MV1523_LSR_times),
    sum(ANN3_10_MV1523_LSR_times)
]

Total_Time_MV1530 = [
    sum(ANN4_100000_MV1530_LSR_times),
    sum(ANN4_50000_MV1530_LSR_times),
    sum(ANN4_10000_MV1530_LSR_times),
    sum(ANN4_1000_MV1530_LSR_times),
    sum(ANN4_100_MV1530_LSR_times),
    sum(ANN4_10_MV1530_LSR_times)
]

In [ ]:
Total_Time_MV1530

Now make a figure to compare computational times. 

In [ ]:
os.chdir(r'XXXX') ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO HAVE THE FIGURES STORED####################

training_data = [100000, 50000, 10000, 1000, 100, 10]
plt.scatter(training_data,Total_Time_MV1519, s=100, edgecolor='black', color='lime' , marker='o', label='MV1519')
plt.scatter(training_data,Total_Time_MV1523, s=100,edgecolor='black', color='red', marker='^', label='MV1523')
plt.scatter(training_data,Total_Time_MV1530, s=100, edgecolor='black', color='purple', marker='s', label='MV1530')
plt.xscale('log')


ax = plt.gca()

# Set the linewidth of the axis lines
ax.spines['top'].set_linewidth(2)
ax.spines['right'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)
plt.yticks([250, 500, 750])  # Specify the desired tick values
plt.ylim(0, 1000)

plt.tick_params(axis='both', which='major', direction='in', width=3, labelsize =18, length=4)  # Adjust 'width' for thickness
plt.tick_params(axis='both', which='minor', direction='in', width=3, labelsize =18, length=4)  # Optionally adjust minor ticks

plt.legend(fontsize=18, frameon=False, loc=(0.6, 0.675))
#plt.title(str(key) + ': ANN_10')
plt.xlabel('Training Set Size', fontsize=18)
plt.ylabel('Computation Time (s)', fontsize=18)


# Get current axes
ax = plt.gca()

# Set the tick marks to point inside
ax.tick_params(axis='both', direction='in', pad = 5, labelsize=18)
plt.savefig("ANN_LSR_Time.jpg", dpi=600, bbox_inches='tight' )

Now lets take the new LSR predictions and calculate the unweighted error.

The next cell re-loads stored data from the previous LSR analysis. This does not need to be used if the varaibles are already defined.

In [ ]:
os.chdir(r'XXXX')  ########## PLEASE PUT THE DIRECTORY WHERE THE ANN + LSR DATA IS STORED IF YOU RUN THIS BLOCK ####################

# Start with MV1519 params
ANN3_10_MV1519_LSR_params = pd.read_csv('ANN3_10_MV1519_LSR_params.csv')
ANN3_100_MV1519_LSR_params = pd.read_csv('ANN3_100_MV1519_LSR_params.csv')
ANN3_1000_MV1519_LSR_params = pd.read_csv('ANN3_1000_MV1519_LSR_params.csv')
ANN3_10000_MV1519_LSR_params = pd.read_csv('ANN3_10000_MV1519_LSR_params.csv')
ANN3_50000_MV1519_LSR_params = pd.read_csv('ANN3_50000_MV1519_LSR_params.csv')
ANN3_100000_MV1519_LSR_params = pd.read_csv('ANN3_100000_MV1519_LSR_params.csv')

# MV1519 times

ANN3_10_MV1519_LSR_times = pd.read_csv( 'ANN3_10_MV1519_LSR_times.csv')['0'].to_numpy()
ANN3_100_MV1519_LSR_times = pd.read_csv( 'ANN3_100_MV1519_LSR_times.csv')['0'].to_numpy()
ANN3_1000_MV1519_LSR_times = pd.read_csv( 'ANN3_1000_MV1519_LSR_times.csv')['0'].to_numpy()
ANN3_10000_MV1519_LSR_times = pd.read_csv( 'ANN3_10000_MV1519_LSR_times.csv')['0'].to_numpy()
ANN3_50000_MV1519_LSR_times = pd.read_csv( 'ANN3_50000_MV1519_LSR_times.csv')['0'].to_numpy()
ANN3_100000_MV1519_LSR_times = pd.read_csv( 'ANN3_100000_MV1519_LSR_times.csv')['0'].to_numpy()

# MV1523 params

ANN3_10_MV1523_LSR_params = pd.read_csv('ANN3_10_MV1523_LSR_params.csv')
ANN3_100_MV1523_LSR_params = pd.read_csv('ANN3_100_MV1523_LSR_params.csv')
ANN3_1000_MV1523_LSR_params = pd.read_csv('ANN3_1000_MV1523_LSR_params.csv')
ANN3_10000_MV1523_LSR_params = pd.read_csv('ANN3_10000_MV1523_LSR_params.csv')
ANN3_50000_MV1523_LSR_params = pd.read_csv('ANN3_50000_MV1523_LSR_params.csv')
ANN3_100000_MV1523_LSR_params = pd.read_csv('ANN3_100000_MV1523_LSR_params.csv')

# MV1523 times

ANN3_10_MV1523_LSR_times = pd.read_csv( 'ANN3_10_MV1523_LSR_times.csv')['0'].to_numpy()
ANN3_100_MV1523_LSR_times = pd.read_csv( 'ANN3_100_MV1523_LSR_times.csv')['0'].to_numpy()
ANN3_1000_MV1523_LSR_times = pd.read_csv( 'ANN3_1000_MV1523_LSR_times.csv')['0'].to_numpy()
ANN3_10000_MV1523_LSR_times = pd.read_csv( 'ANN3_10000_MV1523_LSR_times.csv')['0'].to_numpy()
ANN3_50000_MV1523_LSR_times = pd.read_csv( 'ANN3_50000_MV1523_LSR_times.csv')['0'].to_numpy()
ANN3_100000_MV1523_LSR_times = pd.read_csv( 'ANN3_100000_MV1523_LSR_times.csv')['0'].to_numpy()

# MV1530 params

ANN4_10_MV1530_LSR_params = pd.read_csv('ANN4_10_MV1530_LSR_params.csv')
ANN4_100_MV1530_LSR_params = pd.read_csv('ANN4_100_MV1530_LSR_params.csv')
ANN4_1000_MV1530_LSR_params = pd.read_csv('ANN4_1000_MV1530_LSR_params.csv')
ANN4_10000_MV1530_LSR_params = pd.read_csv('ANN4_10000_MV1530_LSR_params.csv')
ANN4_50000_MV1530_LSR_params = pd.read_csv('ANN4_50000_MV1530_LSR_params.csv')
ANN4_100000_MV1530_LSR_params = pd.read_csv('ANN4_100000_MV1530_LSR_params.csv')

# MV1539 times

ANN4_10_MV1530_LSR_times = pd.read_csv( 'ANN4_10_MV1530_LSR_times.csv')['0'].to_numpy()
ANN4_100_MV1530_LSR_times = pd.read_csv( 'ANN4_100_MV1530_LSR_times.csv')['0'].to_numpy()
ANN4_1000_MV1530_LSR_times = pd.read_csv( 'ANN4_1000_MV1530_LSR_times.csv')['0'].to_numpy()
ANN4_10000_MV1530_LSR_times = pd.read_csv( 'ANN4_10000_MV1530_LSR_times.csv')['0'].to_numpy()
ANN4_50000_MV1530_LSR_times = pd.read_csv( 'ANN4_50000_MV1530_LSR_times.csv')['0'].to_numpy()
ANN4_100000_MV1530_LSR_times = pd.read_csv( 'ANN4_100000_MV1530_LSR_times.csv')['0'].to_numpy()

In [ ]:
MV1519_list_LSR = [
    ANN3_10_MV1519_LSR_params, 
    ANN3_100_MV1519_LSR_params, 
    ANN3_1000_MV1519_LSR_params,
    ANN3_10000_MV1519_LSR_params, 
    ANN3_50000_MV1519_LSR_params, 
    ANN3_100000_MV1519_LSR_params,
]

ANN3_MV1519_LSR_NCS, ANN3_MV1519_LSR_CL = NCS_Genorator_ANN3(MV1519_list_LSR, 1.68, E, wv)

In [ ]:
############ Generating error between ANN3 + LSR and experiment and LSR and experiment ################

path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO HAVE THESE PLOTS STORED ####################

# If you do not want to make plots, set "generate_images=False"
# If you do want to make plots, set "generate_images=True"

MV1519_Sigma_PostLSR, MV1519_LSR_Sigma_PostLSR = NCS_Comparison_ANN3(ANN3_MV1519_LSR_NCS, # simulated spectra from ANN + LSR
                                                                     MV1519[:, :, 1:], # excperimental data
                                                                     LSR_MV1519_NCS,  # traditional LSR data
                                                                     6, # number of fit parameters
                                                                     path, # path to save images
                                                                     generate_images=False, # makes images when true
                                                                     foldername = 'MV1519_PostLSR') # name for the folder created to contain images. 




In [ ]:
MV1523_list_LSR = [
    ANN3_10_MV1523_LSR_params, 
    ANN3_100_MV1523_LSR_params, 
    ANN3_1000_MV1523_LSR_params,
    ANN3_10000_MV1523_LSR_params, 
    ANN3_50000_MV1523_LSR_params, 
    ANN3_50000_MV1523_LSR_params,
]

ANN3_MV1523_LSR_NCS, ANN3_MV1523_LSR_CL = NCS_Genorator_ANN3(MV1523_list_LSR, 1.65, E, wv)

In [ ]:
############ Generating error between ANN3 + LSR and experiment and LSR and experiment ################

path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO HAVE THESE PLOTS STORED ####################

# If you do not want to make plots, set "generate_images=False"
# If you do want to make plots, set "generate_images=True"

MV1523_Sigma_PostLSR, MV1523_LSR_Sigma_PostLSR = NCS_Comparison_ANN3(ANN3_MV1523_LSR_NCS, # simulated spectra from ANN + LSR
                                                                     MV1523[:, :, 1:], # excperimental data
                                                                     LSR_MV1523_NCS,  # traditional LSR data
                                                                     6, # number of fit parameters
                                                                     path, # path to save images
                                                                     generate_images=False, # makes images when true
                                                                     foldername = 'MV1523_PostLSR') # name for the folder created to contain images. 




In [ ]:
MV1530_list_LSR = [
    ANN4_10_MV1530_LSR_params, 
    ANN4_100_MV1530_LSR_params, 
    ANN4_1000_MV1530_LSR_params,
    ANN4_10000_MV1530_LSR_params, 
    ANN4_50000_MV1530_LSR_params, 
    ANN4_50000_MV1530_LSR_params,
]

ANN4_MV1530_LSR_NCS, ANN4_MV1530_LSR_CL =  NCS_Genorator_ANN4(MV1530_list_LSR, E, wv)

In [ ]:
############ Generating error between ANN4 + LSR and experiment and LSR and experiment ################

path = r'XXXX'  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO HAVE THESE PLOTS STORED ####################

# If you do not want to make plots, set "generate_images=False"
# If you do want to make plots, set "generate_images=True"

MV1530_Sigma_PostLSR, MV1530_LSR_Sigma_PostLSR = NCS_Comparison_ANN4_v2(ANN4_MV1530_LSR_NCS,  # simulated spectra from ANN + LSR
                                                                        MV1530[:, :, 1:], # excperimental data
                                                                        LSR_MV1530_NCS, # traditional LSR data
                                                                        7, # number of fit parameters
                                                                        path, # path to save images
                                                                        generate_images=False, # makes images when true
                                                                        foldername = 'MV1530_PostLSR' # name for the folder created to contain images. 
                                                                       )

In [ ]:
data1 = MV1519_Sigma_PostLSR
data1.append(MV1519_LSR_Sigma[0])

data2 = MV1523_Sigma_PostLSR
data2.append(MV1523_LSR_Sigma[0])

data3 = MV1530_Sigma_PostLSR
data3.append(MV1530_LSR_Sigma[0])

In [ ]:
 MV1519_Sigma_PostLSR[1]

The following code displays the sigma values for each data point after the ANN predictions are fed into the least-square regression process

In [ ]:
os.chdir(r'XXXX')  ########## PLEASE PUT THE DIRECTORY WHERE YOU WOULD LIKE TO HAVE THESE PLOTS STORED ####################

font_properties = {'fontsize': 16, 'fontweight': 'normal', 'family': 'sans-serif'}

# Create a figure and axes for the subplots
fig, axs = plt.subplots(3, 1, figsize=(8, 8))

tick_labels = ['$10^{1}$ ', '$10^{2}$', '$10^{3}$' , '$10^{4}$', r'$5 \times 10^{4}$', '$10^{5}$', 'LSR']

# Create box plots for the first dataset
boxplot1 = axs[0].boxplot(data1,       
            vert=True, 
            patch_artist=True, 
            flierprops=dict(marker='', color='red', alpha=0.5),
            boxprops=dict(linewidth=3),
            whiskerprops=dict(linewidth=3), 
            medianprops=dict(color='black', linewidth=3))  # Vertical box plots
axs[0].set_title('(a)', fontsize=18, loc='left', y=0.85,x = 0.01)
axs[0].set_ylabel(r'$σ \times 10^{3}$', fontsize=18, labelpad=10)
axs[0].set_ylim(-0.002, 0.032)
axs[0].set_yticks(np.arange(0.00, 0.031, 0.015))
axs[0].set_xticklabels([])
axs[0].tick_params(axis='both', which='major', labelsize=18)  # Adjust labelsize as needed
#custom_ticks = np.linspace(0.05, 0.03, 6) = 14
#axs[0].set_ylim(-0.005, 0.03)
#axs[0].set_yticks(custom_ticks)


# Create box plots for the second dataset
boxplot2 = axs[1].boxplot(data2,        
            vert=True, 
            patch_artist=True, 
            flierprops=dict(marker='', color='red', alpha=0.5),
            boxprops=dict(linewidth=3),
            whiskerprops=dict(linewidth=3), 
            medianprops=dict(color='black', linewidth=3))
axs[1].set_title('(b)', fontsize=18, loc='left', y=0.85,x = 0.01)
axs[1].set_ylim(-0.002, 0.032)
axs[1].set_yticks(np.arange(0.00, 0.031, 0.015))
axs[1].set_ylabel(r'$σ \times 10^{3}$', fontsize=18)
axs[1].set_xticklabels([])
axs[1].tick_params(axis='both', which='major', labelsize=18)  # Adjust labelsize as needed
#axs[1].set_ylim(-3, 4)

# Create box plots for the third dataset
boxplot3 = axs[2].boxplot(data3,    
            vert=True, 
            patch_artist=True, 
            flierprops=dict(marker='', color='red', alpha=0.5),
            boxprops=dict(linewidth=3),
            whiskerprops=dict(linewidth=3), 
            medianprops=dict(color='black', linewidth=3))
axs[2].set_title('(c)', fontsize=18, loc='left', y=0.85,x = 0.01)
axs[2].set_ylabel(r'$σ \times 10^{3}$', fontsize=18, labelpad=10)
axs[2].set_xticklabels(tick_labels, fontdict=font_properties)
axs[2].tick_params(axis='both', which='major', labelsize=18, pad=5)  # Adjust labelsize as needed
custom_ticks = np.linspace(0.0015, 0.0035, 3) 
axs[2].set_xlabel('ANN training size', fontsize=18)
axs[2].set_yticks(custom_ticks)
axs[2].set_ylim(0.0013, 0.0037)

# Hide the box and whisker lines
for box in boxplot1['boxes']:
    box.set_visible(False)  # Hide the box

for whisker in boxplot1['whiskers']:
    whisker.set_visible(False)  # Hide the whiskers

# Optional: You can still display the median and fliers
for median in boxplot1['medians']:
    median.set_visible(False)  # Keep the median line visible

for flier in boxplot1['fliers']:
    flier.set_visible(False)  # Keep the outliers visible

for cap in boxplot1['caps']:
    cap.set_visible(False)  # Hide the caps

# Hide the box and whisker lines
for box in boxplot2['boxes']:
    box.set_visible(False)  # Hide the box

for whisker in boxplot2['whiskers']:
    whisker.set_visible(False)  # Hide the whiskers

# Optional: You can still display the median and fliers
for median in boxplot2['medians']:
    median.set_visible(False)  # Keep the median line visible

for flier in boxplot2['fliers']:
    flier.set_visible(False)  # Keep the outliers visible

for cap in boxplot2['caps']:
    cap.set_visible(False)  # Hide the caps


# Hide the box and whisker lines
for box in boxplot3['boxes']:
    box.set_visible(False)  # Hide the box

for whisker in boxplot3['whiskers']:
    whisker.set_visible(False)  # Hide the whiskers

# Optional: You can still display the median and fliers
for median in boxplot3['medians']:
    median.set_visible(False)  # Keep the median line visible

for flier in boxplot3['fliers']:
    flier.set_visible(False)  # Keep the outliers visible

for cap in boxplot3['caps']:
    cap.set_visible(False)  # Hide the caps


for ax in axs:
    ax.spines['top'].set_linewidth(2)
    ax.spines['right'].set_linewidth(2)
    ax.spines['left'].set_linewidth(2)
    ax.spines['bottom'].set_linewidth(2)

    # Make ticks point inward
    ax.tick_params(direction='in', width=2)



for i in range(len(data1)):
    axs[0].scatter([i + 1] * len(data1[i]), data1[i], color='red', marker='o', edgecolor = 'black')
    axs[1].scatter([i + 1] * len(data2[i]), data2[i], color='red', marker='o', edgecolor = 'black')
    axs[2].scatter([i + 1] * len(data3[i]), data3[i], color='red', marker='o', edgecolor = 'black')


#for i, label in enumerate(plt.gca().get_xticklabels()):
    #if label.get_text() == 'LSR':
        #label.set_y(label.get_position()[1] - 0.01)  # Move Group A up



def format_ticks(x, _):
    return f'{x * 1000:.1f}'  # Multiply by scale_factor

for ax in axs:
    ax.yaxis.set_major_formatter(FuncFormatter(format_ticks))


# Adjust layout
plt.tight_layout()

# Show the plot
plt.savefig("ANN_Condensed_Post_LSR" + str(i) + ".jpg",  dpi=600, bbox_inches='tight' )
plt.show()

